## Define Basic Field and Term Classes

In [1]:
class field(object):
    def __init__(self, symbol, massDim, lorentz_rank, spinor_rank, spinor_rank_conj):
        self.symbol = symbol #string symbol for field
        self.lorentz_rank = lorentz_rank #int indicating lorentz rank of field
        self.spinor_rank = spinor_rank #int indicating spinor rank of field
        self.spinor_rank_conj = spinor_rank_conj #int indicating spinor rank of field
        self.massDim= massDim #int indicating mass dimension of field
    def info(self):
        return 'symbol: ' + str(self.symbol) \
            + ' massDim: ' + str(self.massDim) \
            + ' lorentz_rank: ' + str(self.lorentz_rank) \
            + ' spinor_rank: ' + str(self.spinor_rank)
    def get_symbol(self):
        return self.symbol
    def get_lorentz_rank(self):
        return self.lorentz_rank
    def get_spinor_rank(self):
        return self.spinor_rank
    def get_spinor_rank_conj(self):
        return self.spinor_rank_conj
    def get_massDim(self):
        return self.massDim
    def __eq__(self, other):
        eq = (self.symbol == other.symbol)
        return eq

## Generate All Multisets of Derivatives and Fields of a Given Mass Dimension

In [2]:

def totalMassDim(field_multiplicities, field_massDims):
    if len(field_multiplicities) != len(field_massDims):
        print('totalMassDim(): Input lists must have same length')
        return None
    n_types = len(field_multiplicities)
    total_massDim = 0
    for i in range(n_types):
        total_massDim += field_multiplicities[i]*field_massDims[i]
    return total_massDim

def totalLorentzRank(field_multiplicities, field_lorentzRanks):
    if len(field_multiplicities) != len(field_massDims):
        print('totalLorentzRank(): Input lists must have same length')
        return None
    n_types = len(field_multiplicities)
    total_lorentzRank = 0
    for i in range(n_types):
        total_lorentzRank += field_multiplicities[i]*field_lorentzRanks[i]
    return total_lorentzRank

def convert_to_dict(field_multiplicities, field_types, remove_zeros):
    # here, field_types = ['D', 'F', 'S', 'V', 'T', 'Vp', 'Sp'], but may be adjusted to include more
    # field types (for example, in theories with multiple Fermi fields or non-Abelian gauge fields) if necessary.
    if len(field_multiplicities) != len(field_types):
        print('Length of field_multiplicities input to convert_to_dict() must be equal to length of field_types.')
        return None
    n_types = len(field_types)
    field_multiset = {}
    if remove_zeros:
        for i in range(n_types):
            if field_multiplicities[i] != 0:
                field_multiset[field_types[i]] = field_multiplicities[i]
        return field_multiset
    else:
        for i in range(n_types):
            field_multiset[field_types[i]] = field_multiplicities[i]
        return field_multiset
    
def generate_field_multisets(massDim, field_symbols, field_massDims, field_lorentzRanks):
    n_types = len(field_symbols)
    zero_list = [0 for i in range(len(field_symbols))]
    front = [zero_list]
    field_multiplicities_dict = {}
    
    counter = 0
    while front:
        #print(counter)
        front_new = front.copy()
        for i in range(len(front)):
            #print('i: ' + str(i))
            multiplicities_list = front[i]
            #print(multiplicities_list)
            # find index last non-zero element
            if multiplicities_list == len(field_symbols)*[0]:
                index_last_nonzero = 0
            else: 
                index_last_nonzero = [i for (i, m) in enumerate(multiplicities_list) if m != 0][-1]
                
            for j in range(index_last_nonzero, n_types):
                multiplicities_list_new = multiplicities_list.copy()
                multiplicities_list_new[j] += 1
                M = totalMassDim(multiplicities_list_new, field_massDims)
                L = totalLorentzRank(multiplicities_list_new, field_lorentzRanks)
                if M < massDim:
                    #print('front_new before append:' + str(front_new))
                    front_new.append(multiplicities_list_new)
                    #print('front_new after append:' + str(front_new))
                if M <= massDim and L%2 == 0:
                    try:
                        field_multiplicities_dict[M].append(multiplicities_list_new)
                    except:
                        field_multiplicities_dict[M] = [multiplicities_list_new]
            #print('len(front): ' + str(len(front)))
            #print('len(front_new): ' + str(len(front_new)))
            #del front_new[i]
            front_new.remove(multiplicities_list)
        front = front_new
        counter += 1
    
    # create field multisets dict, where each value is a dict rather than a list
    field_multisets_dict = {}
    
    for key in field_multiplicities_dict.keys():
        field_multisets_dict[key] = []
        field_multiplicities_list = field_multiplicities_dict[key]
        for field_multiplicities in field_multiplicities_list:
            # if number of derivatives (field_multiplicities[0]) equal to mass dimension (key), do not include
            # since multiset is all derivatives
            if field_multiplicities[0] == key: 
                continue
            field_multiset = convert_to_dict(field_multiplicities, field_symbols, True)
            field_multisets_dict[key].append(field_multiset)   
    return field_multisets_dict


In [3]:
massDim = 6
field_symbols = ['D', 'F', 'S', 'V', 'T', 'Vp', 'Sp']
field_massDims = [1, 2, 3, 3, 3 ,3 ,3]
field_lorentzRanks = [1, 2, 0, 1, 2, 1, 0]
field_multisets_dict = generate_field_multisets(massDim, field_symbols, field_massDims, field_lorentzRanks)
for (key, value) in field_multisets_dict.items():
    print(key)
    print(value)
    #print('len(field_multisets_dict[' + str(key) + ']): ' + str(len(value)))
    

2
[{'F': 1}]
3
[{'S': 1}, {'T': 1}, {'Sp': 1}]
4
[{'D': 1, 'V': 1}, {'D': 1, 'Vp': 1}, {'F': 2}, {'D': 2, 'F': 1}]
5
[{'F': 1, 'S': 1}, {'F': 1, 'T': 1}, {'F': 1, 'Sp': 1}, {'D': 2, 'S': 1}, {'D': 2, 'T': 1}, {'D': 2, 'Sp': 1}]
6
[{'S': 2}, {'S': 1, 'T': 1}, {'S': 1, 'Sp': 1}, {'V': 2}, {'V': 1, 'Vp': 1}, {'T': 2}, {'T': 1, 'Sp': 1}, {'Vp': 2}, {'Sp': 2}, {'D': 1, 'F': 1, 'V': 1}, {'D': 1, 'F': 1, 'Vp': 1}, {'F': 3}, {'D': 3, 'V': 1}, {'D': 3, 'Vp': 1}, {'D': 2, 'F': 2}, {'D': 4, 'F': 1}]


In [4]:
massDim = 6
field_symbols = ['D', 'F', 'S', 'V', 'T', 'Vp', 'Sp']
field_massDims = [1, 2, 3, 3, 3 ,3 ,3]
field_lorentzRanks = [1, 2, 0, 1, 2, 1, 0]
field_multisets_dict = generate_field_multisets(massDim, field_symbols, field_massDims, field_lorentzRanks)
for (key, value) in field_multisets_dict.items():
    print(key)
    print(value)
    #print('len(field_multisets_dict[' + str(key) + ']): ' + str(len(value)))

2
[{'F': 1}]
3
[{'S': 1}, {'T': 1}, {'Sp': 1}]
4
[{'D': 1, 'V': 1}, {'D': 1, 'Vp': 1}, {'F': 2}, {'D': 2, 'F': 1}]
5
[{'F': 1, 'S': 1}, {'F': 1, 'T': 1}, {'F': 1, 'Sp': 1}, {'D': 2, 'S': 1}, {'D': 2, 'T': 1}, {'D': 2, 'Sp': 1}]
6
[{'S': 2}, {'S': 1, 'T': 1}, {'S': 1, 'Sp': 1}, {'V': 2}, {'V': 1, 'Vp': 1}, {'T': 2}, {'T': 1, 'Sp': 1}, {'Vp': 2}, {'Sp': 2}, {'D': 1, 'F': 1, 'V': 1}, {'D': 1, 'F': 1, 'Vp': 1}, {'F': 3}, {'D': 3, 'V': 1}, {'D': 3, 'Vp': 1}, {'D': 2, 'F': 2}, {'D': 4, 'F': 1}]


In [5]:
massDim = 4
field_symbols = ['A', 'B', 'C']
field_massDims = [1, 2, 3]
field_lorentzRanks = [1, 2, 1]
field_multisets_dict = generate_field_multisets(massDim, field_symbols, field_massDims, field_lorentzRanks)
for (key, value) in field_multisets_dict.items():
    print(key)
    print(value)
    print('len(field_multisets_dict[m]): ' + str(len(value)))

2
[{'B': 1}]
len(field_multisets_dict[m]): 1
4
[{'A': 1, 'C': 1}, {'B': 2}, {'A': 2, 'B': 1}]
len(field_multisets_dict[m]): 3


## Generate All Possible Derivative Assignments for a Given Multiset of Fields and Derivatives

In [6]:
# tuples_sum() implementation taken from http://code.activestate.com/recipes/578608-generates-tuples-of-integers-with-a-given-sum/
from itertools import permutations

def tuples_sum(nbval,total,order=True) :
    """ 
        Generate all the tuples L of nbval positive or nul integer 
        such that sum(L)=total.
        The tuples may be ordered (decreasing order) or not
    """
    if nbval == 0 and total == 0 : yield tuple() ; return #raise StopIteration
    if nbval == 1 : yield (total,) ; return #raise StopIteration
    if total==0 : yield (0,)*nbval ; return #raise StopIteration
    for start in range(total,0,-1) :
        for qu in tuples_sum(nbval-1,total-start) :
            if qu[0]<=start :
                sol=(start,)+qu
                if order : yield sol
                else :
                    l=set()
                    for p in permutations(sol,len(sol)) :
                        if p not in l :
                            l.add(p)
                            yield p
    
if __name__=='__main__' :
    print("How to obtain 5 by adding a+b+c (>=0) ? ")
    print("Give me the list of (a,b,c) tuples.")
    g=tuples_sum(3,6,order=False)
    print(list(g))
    
    print("How to obtain 6 by adding 3 positive or nul integers ?")
    g=tuples_sum(3,6,order=True)
    print(list(g))

How to obtain 5 by adding a+b+c (>=0) ? 
Give me the list of (a,b,c) tuples.
[(6, 0, 0), (0, 6, 0), (0, 0, 6), (5, 1, 0), (5, 0, 1), (1, 5, 0), (1, 0, 5), (0, 5, 1), (0, 1, 5), (4, 2, 0), (4, 0, 2), (2, 4, 0), (2, 0, 4), (0, 4, 2), (0, 2, 4), (4, 1, 1), (1, 4, 1), (1, 1, 4), (3, 3, 0), (3, 0, 3), (0, 3, 3), (3, 2, 1), (3, 1, 2), (2, 3, 1), (2, 1, 3), (1, 3, 2), (1, 2, 3), (2, 2, 2)]
How to obtain 6 by adding 3 positive or nul integers ?
[(6, 0, 0), (5, 1, 0), (4, 2, 0), (4, 1, 1), (3, 3, 0), (3, 2, 1), (2, 2, 2)]


In [7]:
n = 4
m = 7
list(tuples_sum(m,n,order=False))


[(4, 0, 0, 0, 0, 0, 0),
 (0, 4, 0, 0, 0, 0, 0),
 (0, 0, 4, 0, 0, 0, 0),
 (0, 0, 0, 4, 0, 0, 0),
 (0, 0, 0, 0, 4, 0, 0),
 (0, 0, 0, 0, 0, 4, 0),
 (0, 0, 0, 0, 0, 0, 4),
 (3, 1, 0, 0, 0, 0, 0),
 (3, 0, 1, 0, 0, 0, 0),
 (3, 0, 0, 1, 0, 0, 0),
 (3, 0, 0, 0, 1, 0, 0),
 (3, 0, 0, 0, 0, 1, 0),
 (3, 0, 0, 0, 0, 0, 1),
 (1, 3, 0, 0, 0, 0, 0),
 (1, 0, 3, 0, 0, 0, 0),
 (1, 0, 0, 3, 0, 0, 0),
 (1, 0, 0, 0, 3, 0, 0),
 (1, 0, 0, 0, 0, 3, 0),
 (1, 0, 0, 0, 0, 0, 3),
 (0, 3, 1, 0, 0, 0, 0),
 (0, 3, 0, 1, 0, 0, 0),
 (0, 3, 0, 0, 1, 0, 0),
 (0, 3, 0, 0, 0, 1, 0),
 (0, 3, 0, 0, 0, 0, 1),
 (0, 1, 3, 0, 0, 0, 0),
 (0, 1, 0, 3, 0, 0, 0),
 (0, 1, 0, 0, 3, 0, 0),
 (0, 1, 0, 0, 0, 3, 0),
 (0, 1, 0, 0, 0, 0, 3),
 (0, 0, 3, 1, 0, 0, 0),
 (0, 0, 3, 0, 1, 0, 0),
 (0, 0, 3, 0, 0, 1, 0),
 (0, 0, 3, 0, 0, 0, 1),
 (0, 0, 1, 3, 0, 0, 0),
 (0, 0, 1, 0, 3, 0, 0),
 (0, 0, 1, 0, 0, 3, 0),
 (0, 0, 1, 0, 0, 0, 3),
 (0, 0, 0, 3, 1, 0, 0),
 (0, 0, 0, 3, 0, 1, 0),
 (0, 0, 0, 3, 0, 0, 1),
 (0, 0, 0, 1, 3, 0, 0),
 (0, 0, 0, 1, 0,

In [8]:
m = 2
n = 5
list(tuples_sum(m,n,order=False))
   

[(5, 0), (0, 5), (4, 1), (1, 4), (3, 2), (2, 3)]

In [9]:
#import inspect # this is to check recursion depth for debugging

def generate_pair_partitions(inner_partition, i_start):
    # INPUT:
    # inner_partition: sorted list (non-increasing order) of numbers that sum to some n_Di, an element of the outer
    # partition of n_D. 
    # OUTPUT:
    # pair_partitions_list: a list of lists of tuples, where each tuple has two elements that sum to the corresponding
    # element of the inner_partition. The number of tuples in each sublist matches the number of elements in 
    # inner_partition. 
    # EXPLANATION:
    # recursive function that returns a list of all distinct ways to partition elements of inner partition into two
    # non-negative integers (indicating number of derivatives acting on psi_bar and psi, respectively). 

    # base case
    #print('RECURSION DEPTH: ' + str(len(inspect.stack(0))-29))
    if len(inner_partition) == 1:
        pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))[i_start:]
        pair_partitions_list = [[item] for item in pair_partitions]
        return pair_partitions_list
    
    # if two successive elements of inner_partition are equal, then different orderings of 
    # corresponding pair partitions [..., (a,b), (c,d), ...] and [..., (c,d), (a,b), ...] are equal. To avoid
    # repeats, need to modify recursion step for case when successive elements are equal. 
    if inner_partition[1] == inner_partition[0]:
        pair_partitions_list = []
        pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))
        for i in range(i_start, len(pair_partitions)):
            pair_partitions_list_old = generate_pair_partitions(inner_partition[1:], i)
            #print('RECURSION DEPTH: ' + str(len(inspect.stack(0))-29))
            extension = pair_partitions[i]
            for j in range(len(pair_partitions_list_old)):
                pair_partition_old = pair_partitions_list_old[j]
                pair_partition_new = [extension] + pair_partition_old 
                pair_partitions_list.append(pair_partition_new)
        return pair_partitions_list
    
    # recursion step is simpler in case where successive elements are not equal
    else:
        pair_partitions_list = []
        pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))[i_start:]
        pair_partitions_list_old = generate_pair_partitions(inner_partition[1:], 0)
        for i in range(len(pair_partitions)):
            extension = pair_partitions[i]
            for j in range(len(pair_partitions_list_old)):
                pair_partition_old = pair_partitions_list_old[j]
                pair_partition_new = [extension] + pair_partition_old 
                pair_partitions_list.append(pair_partition_new)
        return pair_partitions_list
    


In [10]:
inner_partition = [3, 2, 2, 1]
generate_pair_partitions(inner_partition, 0)

[[(3, 0), (2, 0), (2, 0), (1, 0)],
 [(3, 0), (2, 0), (2, 0), (0, 1)],
 [(3, 0), (2, 0), (0, 2), (1, 0)],
 [(3, 0), (2, 0), (0, 2), (0, 1)],
 [(3, 0), (2, 0), (1, 1), (1, 0)],
 [(3, 0), (2, 0), (1, 1), (0, 1)],
 [(3, 0), (0, 2), (0, 2), (1, 0)],
 [(3, 0), (0, 2), (0, 2), (0, 1)],
 [(3, 0), (0, 2), (1, 1), (1, 0)],
 [(3, 0), (0, 2), (1, 1), (0, 1)],
 [(3, 0), (1, 1), (1, 1), (1, 0)],
 [(3, 0), (1, 1), (1, 1), (0, 1)],
 [(0, 3), (2, 0), (2, 0), (1, 0)],
 [(0, 3), (2, 0), (2, 0), (0, 1)],
 [(0, 3), (2, 0), (0, 2), (1, 0)],
 [(0, 3), (2, 0), (0, 2), (0, 1)],
 [(0, 3), (2, 0), (1, 1), (1, 0)],
 [(0, 3), (2, 0), (1, 1), (0, 1)],
 [(0, 3), (0, 2), (0, 2), (1, 0)],
 [(0, 3), (0, 2), (0, 2), (0, 1)],
 [(0, 3), (0, 2), (1, 1), (1, 0)],
 [(0, 3), (0, 2), (1, 1), (0, 1)],
 [(0, 3), (1, 1), (1, 1), (1, 0)],
 [(0, 3), (1, 1), (1, 1), (0, 1)],
 [(2, 1), (2, 0), (2, 0), (1, 0)],
 [(2, 1), (2, 0), (2, 0), (0, 1)],
 [(2, 1), (2, 0), (0, 2), (1, 0)],
 [(2, 1), (2, 0), (0, 2), (0, 1)],
 [(2, 1), (2, 0), (1

In [11]:
inner_partition = [3, 3, 2, 2]
generate_pair_partitions(inner_partition, 0)

[[(3, 0), (3, 0), (2, 0), (2, 0)],
 [(3, 0), (3, 0), (2, 0), (0, 2)],
 [(3, 0), (3, 0), (2, 0), (1, 1)],
 [(3, 0), (3, 0), (0, 2), (0, 2)],
 [(3, 0), (3, 0), (0, 2), (1, 1)],
 [(3, 0), (3, 0), (1, 1), (1, 1)],
 [(3, 0), (0, 3), (2, 0), (2, 0)],
 [(3, 0), (0, 3), (2, 0), (0, 2)],
 [(3, 0), (0, 3), (2, 0), (1, 1)],
 [(3, 0), (0, 3), (0, 2), (0, 2)],
 [(3, 0), (0, 3), (0, 2), (1, 1)],
 [(3, 0), (0, 3), (1, 1), (1, 1)],
 [(3, 0), (2, 1), (2, 0), (2, 0)],
 [(3, 0), (2, 1), (2, 0), (0, 2)],
 [(3, 0), (2, 1), (2, 0), (1, 1)],
 [(3, 0), (2, 1), (0, 2), (0, 2)],
 [(3, 0), (2, 1), (0, 2), (1, 1)],
 [(3, 0), (2, 1), (1, 1), (1, 1)],
 [(3, 0), (1, 2), (2, 0), (2, 0)],
 [(3, 0), (1, 2), (2, 0), (0, 2)],
 [(3, 0), (1, 2), (2, 0), (1, 1)],
 [(3, 0), (1, 2), (0, 2), (0, 2)],
 [(3, 0), (1, 2), (0, 2), (1, 1)],
 [(3, 0), (1, 2), (1, 1), (1, 1)],
 [(0, 3), (0, 3), (2, 0), (2, 0)],
 [(0, 3), (0, 3), (2, 0), (0, 2)],
 [(0, 3), (0, 3), (2, 0), (1, 1)],
 [(0, 3), (0, 3), (0, 2), (0, 2)],
 [(0, 3), (0, 3), (0

In [12]:
inner_partition = [2, 2, 1]
generate_pair_partitions(inner_partition, 0)

[[(2, 0), (2, 0), (1, 0)],
 [(2, 0), (2, 0), (0, 1)],
 [(2, 0), (0, 2), (1, 0)],
 [(2, 0), (0, 2), (0, 1)],
 [(2, 0), (1, 1), (1, 0)],
 [(2, 0), (1, 1), (0, 1)],
 [(0, 2), (0, 2), (1, 0)],
 [(0, 2), (0, 2), (0, 1)],
 [(0, 2), (1, 1), (1, 0)],
 [(0, 2), (1, 1), (0, 1)],
 [(1, 1), (1, 1), (1, 0)],
 [(1, 1), (1, 1), (0, 1)]]

In [13]:
inner_partition = [2, 2]
generate_pair_partitions(inner_partition, 0)

[[(2, 0), (2, 0)],
 [(2, 0), (0, 2)],
 [(2, 0), (1, 1)],
 [(0, 2), (0, 2)],
 [(0, 2), (1, 1)],
 [(1, 1), (1, 1)]]

In [14]:
inner_partition = (2, 2, 2)
generate_pair_partitions(inner_partition, 0)

[[(2, 0), (2, 0), (2, 0)],
 [(2, 0), (2, 0), (0, 2)],
 [(2, 0), (2, 0), (1, 1)],
 [(2, 0), (0, 2), (0, 2)],
 [(2, 0), (0, 2), (1, 1)],
 [(2, 0), (1, 1), (1, 1)],
 [(0, 2), (0, 2), (0, 2)],
 [(0, 2), (0, 2), (1, 1)],
 [(0, 2), (1, 1), (1, 1)],
 [(1, 1), (1, 1), (1, 1)]]

In [118]:
def generate_extended_pair_partitions(inner_partitions_list, first_is_F):
    # INPUT:
    # inner_partitions_list: list of sorted tuples (each in non-increasing order) where each tuple has m_i positive
        # integers that sum to n_i, where the n_i sum to n, the total number of derivatives. 
    # first_is_F: indicates whether first sublist of inner_partitions_list corresponds to F-type fields and so 
        # does not need to be further partitioned, by contrast with bilinear fields. 
    # OUTPUT:
    # extended_pair_partitions_list: list of lists of m_i-tuples of 2-tuples, each sublist an extended pair 
    # partition, only represents Dirac bilinear fields, not F fields. 
    # EXPLANATION:
    # recursively generates a list of all sequences (lists) of pair partitions, where each pair partition is an m_i-tuple of 
    # 2-tuples, one m_i-tuple for each fielnd type. each sequence contains one pair partition for each
    # of the tuples in inner_partitions_list (one such tuple serves as an argument for generate_pair_partitions()).
    # if first inner partition in inner_partition_list corresponds to distribution of derivatives among F-type fields, omit
    # first partition. NOTE: output lists do not include numbers of derivatives acting on F-type fields. Full 
    # derivative assignment is output by generate_full_partitions(inner_partitions_list, first_is_F). 
    
    # remove any inner partition for F-type fields to focus on pair partitions of bilinear fields
    if first_is_F == True:
        inner_partitions_list = inner_partitions_list[1:]
        # if remaining list of inner partitions is empty, return empty list
        if len(inner_partitions_list)==0: 
            return []
    
    # base case
    if len(inner_partitions_list) == 1:
        partition = inner_partitions_list[0]
        pair_partitions_list = generate_pair_partitions(partition, 0)
        extended_pair_partitions_list = [[pair_partitions_list[i]] for i in range(len(pair_partitions_list))]
        return extended_pair_partitions_list
    
    # main body
    extended_pair_partitions_list = []
    first_partition = inner_partitions_list[0]
    first_pair_partitions_list = generate_pair_partitions(first_partition, 0) # list of all pair partitions of first inner partition
    extended_pair_partitions_list_old = generate_extended_pair_partitions(inner_partitions_list[1:], False)
    for i in range(len(first_pair_partitions_list)):
        extension = first_pair_partitions_list[i]
        for j in range(len(extended_pair_partitions_list_old)):
            extended_pair_partition_old = extended_pair_partitions_list_old[j]
            extended_pair_partition_new = [extension] + extended_pair_partition_old 
            extended_pair_partitions_list.append(extended_pair_partition_new)
    
    return extended_pair_partitions_list   
        

In [23]:
inner_partitions_list = [[3, 2, 2, 1], [3, 3]]
generate_extended_pair_partitions(inner_partitions_list, True)

[[[(3, 0), (3, 0)]],
 [[(3, 0), (0, 3)]],
 [[(3, 0), (2, 1)]],
 [[(3, 0), (1, 2)]],
 [[(0, 3), (0, 3)]],
 [[(0, 3), (2, 1)]],
 [[(0, 3), (1, 2)]],
 [[(2, 1), (2, 1)]],
 [[(2, 1), (1, 2)]],
 [[(1, 2), (1, 2)]]]

In [136]:
def generate_full_partitions(inner_partitions_list, first_is_F):
    # INPUT:
    # inner_partitions_list: list of sorted tuples (each in non-increasing order) where each tuple has m_i positive
        # integers that sum to n_i, where the n_i sum to n, the total number of derivatives. 
    # first_is_F: indicates whether first sublist of inner_partitions_list corresponds to F-type fields and so 
        # does not need to be further partitioned, by contrast with bilinear fields. 
    # OUTPUT:
    # full_partitions_list: list of lists of m_i-tuples. if first_is_F is True, first sublist of full_partitions_list
    # is equal to first sublist of inner_partitions_list. remaining sublists of full_partitions_list are lists of 
    # 2-tuples, each sublist a pair partition for a Dirac bilinear.
    if first_is_F:
        F_inner_partition = inner_partitions_list[0]
        full_partitions_list = []
        if len(inner_partitions_list) > 1: # if inner_partitions_list contains more than just F fields
            extended_pair_partitions_list = generate_extended_pair_partitions(inner_partitions_list, first_is_F)
            for pair_partitions_list in extended_pair_partitions_list:
                full_partition = [list(F_inner_partition)] + pair_partitions_list
                full_partitions_list.append(full_partition)
        elif len(inner_partitions_list)==1:
            full_partition = [list(F_inner_partition)]
            full_partitions_list.append(full_partition)
        else: 
            return []
        return full_partitions_list
    else:
        # if first sublist inner_partitions_list is not for F-type fields (because these are absent), simply
        # return the pair partitions output by generate_extended_pair_partitions()
        full_partitions_list = generate_extended_pair_partitions(inner_partitions_list, first_is_F)
        return full_partitions_list
        

In [137]:
inner_partitions_list = [[3, 2, 2, 1], [3, 3]]
generate_full_partitions(inner_partitions_list, True)

[[[3, 2, 2, 1], [(3, 0), (3, 0)]],
 [[3, 2, 2, 1], [(3, 0), (0, 3)]],
 [[3, 2, 2, 1], [(3, 0), (2, 1)]],
 [[3, 2, 2, 1], [(3, 0), (1, 2)]],
 [[3, 2, 2, 1], [(0, 3), (0, 3)]],
 [[3, 2, 2, 1], [(0, 3), (2, 1)]],
 [[3, 2, 2, 1], [(0, 3), (1, 2)]],
 [[3, 2, 2, 1], [(2, 1), (2, 1)]],
 [[3, 2, 2, 1], [(2, 1), (1, 2)]],
 [[3, 2, 2, 1], [(1, 2), (1, 2)]]]

In [138]:
def generate_inner_partitions(outer_partition, field_multiplicities):
    # INPUT:
    # outer_partition: list of numbers that sum to number of derivatives n_D with n_fields elements. 
    # field_multiplicities: list of multiplicities of fields, same length as outer_partition
    # OUTPUT:
    # subpartitions_list: list of lists of tuples, with each tuple a sorted partition of the corresponding element
    # of outer partition
    # EXPLANATION:
    # recursive function that returns a list of all distinct ways to sub-partition elements of outer partition.
    # if there are k_1 distinct ways to partition n1, k_2 ways to partition n2, ..., k_d ways to partition nd,
    # then there are k_1 x k_2 x ... x k_d distinct ways to sub-partition elements of outer partition. 

    if len(outer_partition) != len(field_multiplicities):
        print('Error: Lengths of outer_partition and field_multiplicities arguments must be equal.')
        return None

    # base case
    if len(outer_partition)==1:
        n = outer_partition[0]
        m = field_multiplicities[0]
        return [[subpartition_tuple] for subpartition_tuple in tuples_sum(m, n, order=True)]

    # main body
    subpartitions_list = [] # stores all distinct subpartitions 
    n = outer_partition[0]
    m = field_multiplicities[0]
    inner_partitions = list(tuples_sum(m, n, order=True)) 
    subpartitions = generate_inner_partitions(outer_partition[1:], field_multiplicities[1:])
    for i in range(len(inner_partitions)):
        for j in range(len(subpartitions)):
            subpartition_extended = [inner_partitions[i]] + subpartitions[j] 
            subpartitions_list.append(subpartition_extended)

    return subpartitions_list  

In [139]:
outer_partition = [1, 5, 2]
field_multiplicities = [3, 4, 3]
generate_inner_partitions(outer_partition, field_multiplicities)

[[(1, 0, 0), (5, 0, 0, 0), (2, 0, 0)],
 [(1, 0, 0), (5, 0, 0, 0), (1, 1, 0)],
 [(1, 0, 0), (4, 1, 0, 0), (2, 0, 0)],
 [(1, 0, 0), (4, 1, 0, 0), (1, 1, 0)],
 [(1, 0, 0), (3, 2, 0, 0), (2, 0, 0)],
 [(1, 0, 0), (3, 2, 0, 0), (1, 1, 0)],
 [(1, 0, 0), (3, 1, 1, 0), (2, 0, 0)],
 [(1, 0, 0), (3, 1, 1, 0), (1, 1, 0)],
 [(1, 0, 0), (2, 2, 1, 0), (2, 0, 0)],
 [(1, 0, 0), (2, 2, 1, 0), (1, 1, 0)],
 [(1, 0, 0), (2, 1, 1, 1), (2, 0, 0)],
 [(1, 0, 0), (2, 1, 1, 1), (1, 1, 0)]]

In [667]:
def generate_derivative_assignments(field_multiset):
    # INPUT:
    # field_multiset: encodes the set of fields and derivatives making up the term
    #     e.g., field_multiset = {D:3, F:2, S:1, T:2};
    #     if a field is absent, simply omit it - e.g., write {D:3, F:2, S:1, T:2}, not {D:3, F:2, S:1, T:2, V:0}
    # OUTPUT:
    # derivative_assignments_list: A dictionary of lists of lists of tuples, where each sublist of each list is a 
    # distinct assignment of derivatives to fields.
    # EXPLANATION: generates all distinct ways of assigning derivatives to fields in field_multiset. The fields
    # S, V, T, Vp, Sp all contain two fields, a psi and a psi_bar. A derivative may be placed on either psi or psi_bar. 
    # More specifically, generate all 'outer partitions' among the field types - in the above example, F, psi_bar_S, psi_S, psi_bar_T, psi_T.
    # For each outer partition, generate all 'inner_partitions' or 'subpartitions.' Given the 
    # outer partition of n, (n1, n2, n3, n4, n5), where each of the five fields fi occurs with multiplicity mi, a 
    # subpartition is a list of tuples [(), (), (), (), ()], where the ith tuple is a sorted partition of ni with
    # mi elements. The function generates all such lists of tuples
    
    #print(field_multiset)
    # extract number of derivatives
    if 'D' in field_multiset.keys():
        n_D = field_multiset['D']
    else: 
        n_D = 0
    
    # find length of outer partition 
    #bilinear_keys = [k for k in list(field_multiset.keys()) if k != 'D' and k != 'F'] # keys for bilinear elements
    #bilinear_multiplicities = [field_multiset[m] for m in list(field_multiset.keys()) if m != 'D' and m != 'F']
    #n_fields = sum([1 if 'F' in field_multiset.keys() else 0]) \ # use list comprehension on list of one element 
    #    + sum([2 for key in bilinears]) # group psi and psi_bar together as one field for now
    
    #if 'F' in field_multiset.keys():
    #    F_multiplicity = [field_multiset['F']]
    #else: 
    #    F_multiplicity = []
    #bilinear_multiplicities = [x for pair in zip(bilinear_multiplicities,bilinear_multiplicities) for x in pair] 
    # for each bilinear, multiplicities are the same for psi and psi_bar
    field_multiplicities = [field_multiset[x] for x in field_multiset.keys() if x != 'D']
    
    n_fields = len(field_multiplicities)
    #print('n_fields: ' + str(n_fields))
    #print('bilinear_multiplicities: ' + str(bilinear_multiplicities))
    #print('field_multiplicities: ' + str(field_multiplicities))
    
    
    #list of lists, each sublist an outer partition, where different orderings are distinct
    outer_partitions_list = list(tuples_sum(n_fields, n_D, order=False)) 
    
    # for each outer partition, generate a list of sorted inner partitions (ordering does not matter, so one may 
    # as well sort them) where the sum of each inner partition is the corresponding element of the outer partition.
    # this is a list of lists of lists.
    #inner_partitions_dict = {}
    derivative_assignments_dict = {}
    for i in range(len(outer_partitions_list)):
        outer_partition = outer_partitions_list[i]
        #print('outer_partition: ' + str(outer_partition))
        inner_partitions_list = generate_inner_partitions(outer_partition, field_multiplicities)
        #inner_partitions_dict[outer_partition] = inner_partitions_list
        # for each outer partition and inner partition, generate a list of pair partitions
        inner_partitions_dict = {}
        for j in range(len(inner_partitions_list)):
            inner_partition = tuple(inner_partitions_list[j])
            #print('inner_partition: ' + str(inner_partition))
            full_partitions_list = generate_full_partitions(inner_partition, 'F' in field_multiset.keys())
            inner_partitions_dict[inner_partition] = full_partitions_list
        derivative_assignments_dict[outer_partition] = inner_partitions_dict

    return derivative_assignments_dict

Note that in cases where there is an $F$ field present, listed pair partitions below correspond only to elements of the corresponding inner partition after the first, since $F$ fields cannot be split into $\bar{\psi}$ and $\psi$ components. 

In [141]:
field_multiset = {'D':2, 'F': 1, 'S':1, 'V': 2}
derivative_assignments_dict = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments_dict.keys():
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_dict[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_dict[key1][key2]:
            print(value)


outer_partition: (2, 0, 0)
inner_partition: ((2,), (0,), (0, 0))
[[2], [(0, 0)], [(0, 0), (0, 0)]]

outer_partition: (0, 2, 0)
inner_partition: ((0,), (2,), (0, 0))
[[0], [(2, 0)], [(0, 0), (0, 0)]]
[[0], [(0, 2)], [(0, 0), (0, 0)]]
[[0], [(1, 1)], [(0, 0), (0, 0)]]

outer_partition: (0, 0, 2)
inner_partition: ((0,), (0,), (2, 0))
[[0], [(0, 0)], [(2, 0), (0, 0)]]
[[0], [(0, 0)], [(0, 2), (0, 0)]]
[[0], [(0, 0)], [(1, 1), (0, 0)]]
inner_partition: ((0,), (0,), (1, 1))
[[0], [(0, 0)], [(1, 0), (1, 0)]]
[[0], [(0, 0)], [(1, 0), (0, 1)]]
[[0], [(0, 0)], [(0, 1), (0, 1)]]

outer_partition: (1, 1, 0)
inner_partition: ((1,), (1,), (0, 0))
[[1], [(1, 0)], [(0, 0), (0, 0)]]
[[1], [(0, 1)], [(0, 0), (0, 0)]]

outer_partition: (1, 0, 1)
inner_partition: ((1,), (0,), (1, 0))
[[1], [(0, 0)], [(1, 0), (0, 0)]]
[[1], [(0, 0)], [(0, 1), (0, 0)]]

outer_partition: (0, 1, 1)
inner_partition: ((0,), (1,), (1, 0))
[[0], [(1, 0)], [(1, 0), (0, 0)]]
[[0], [(1, 0)], [(0, 1), (0, 0)]]
[[0], [(0, 1)], [(1, 0

In [142]:
#field_multiset = {'D':4, 'F':1, 'S':1, 'V': 2}
field_multiset = {'D':5, 'F':3, 'S':1, 'V': 2}
derivative_assignments_dict = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments_dict.keys():
    print('')
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_dict[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_dict[key1][key2]:
            print(value)



outer_partition: (5, 0, 0)
inner_partition: ((5, 0, 0), (0,), (0, 0))
[[5, 0, 0], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((4, 1, 0), (0,), (0, 0))
[[4, 1, 0], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((3, 2, 0), (0,), (0, 0))
[[3, 2, 0], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((3, 1, 1), (0,), (0, 0))
[[3, 1, 1], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((2, 2, 1), (0,), (0, 0))
[[2, 2, 1], [(0, 0)], [(0, 0), (0, 0)]]


outer_partition: (0, 5, 0)
inner_partition: ((0, 0, 0), (5,), (0, 0))
[[0, 0, 0], [(5, 0)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(0, 5)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(4, 1)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(1, 4)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(3, 2)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(2, 3)], [(0, 0), (0, 0)]]


outer_partition: (0, 0, 5)
inner_partition: ((0, 0, 0), (0,), (5, 0))
[[0, 0, 0], [(0, 0)], [(5, 0), (0, 0)]]
[[0, 0, 0], [(0, 0)], [(0, 5), (0, 0)]]
[[0, 0, 0], [(0, 0)], [(4, 1), (0, 0)]]
[[0, 0, 0], [(0, 0)], [(1, 4), (0, 0)]]
[[0, 0, 0], 

In case below, note that there is no $F$ field, so the number of lists in the pair partition matches the number of elements of the corresponding inner partition.

In [143]:
field_multiset = {'D':3, 'S':1, 'V': 2, 'T': 3}
derivative_assignments_dict = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments_dict.keys():
    print('')
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_dict[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_dict[key1][key2]:
            print(value)



outer_partition: (3, 0, 0)
inner_partition: ((3,), (0, 0), (0, 0, 0))
[[(3, 0)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 3)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(2, 1)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(1, 2)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]


outer_partition: (0, 3, 0)
inner_partition: ((0,), (3, 0), (0, 0, 0))
[[(0, 0)], [(3, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(0, 3), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(2, 1), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(1, 2), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
inner_partition: ((0,), (2, 1), (0, 0, 0))
[[(0, 0)], [(2, 0), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(2, 0), (0, 1)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(0, 2), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(0, 2), (0, 1)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(1, 1), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(1, 1), (0, 1)], [(0, 0), (0, 0), (0, 0)]]


outer_partition: (0, 0, 3)
inner_partitio

## Reducing the List of Derivative Assignments with IBP

In [113]:
from copy import deepcopy

def IBP_remove(derivative_assignment):
    # INPUT: 
    # derivative_assignment: list of lists, each sublist corresponding to a different field type. if F-type
    # fields are represented, first sublist should be a list of integers. any other lists, representing Dirac
    # bilinears, should be lists of 2-tuples.
    # OUTPUT: 
    # remove_bool: True if derivative_assignment meets requirements for removal via IBP, False otherwise
    # EXPLANATION: 
    # checks whether a derivative assignment is to be removed under the specified prescription - i.e.,
    # whether the maximum number of derivatives acting on any field is unique, and whether any field that comes 
    # earlier in the ordering of field types has exactly one fewer derivative acting on it.
    
    # flatten derivative assignment list to facilitate checks below
    first_is_F = type(derivative_assignment[0][0])==int
    if first_is_F:
        flattened_list_F = [x for x in derivative_assignment[0]]
        if len(derivative_assignment) > 1: # if both Dirac bilinears and F fields are included
            flattened_list_bilinears = [x for inner_partition in derivative_assignment[1:] for pair in inner_partition for x in pair]
            flattened_list = flattened_list_F + flattened_list_bilinears
        else: # if only F-type fields are included
            flattened_list = flattened_list_F
    else: # if only Dirac bilinear fields are included
        flattened_list = [x for inner_partition in derivative_assignment for pair in inner_partition for x in pair]
    
    
    # check that maximal number of derivatives only acts on one field
    max_derivs = max(flattened_list)
    unique_max = sum([x == max_derivs for x in flattened_list]) == 1
    #print('unique_max: ' + str(unique_max ))
    
    # check that no field before this field has one fewer derivative acting on it
    max_index = flattened_list.index(max_derivs)
    #print('max_index: ' + str(max_index))
    if max_index > 0:
        one_less_before = max(flattened_list[:max_index]) == max_derivs - 1
    else:
        one_less_before = False
    #print('one_less_before: ' + str(one_less_before))
    
    if unique_max and not one_less_before:
        return True
    
    return False


def IBP_reduce(field_multiset):
    # INPUT:
    # field_multiset: dictionary containing multiplicities of derivative and field types
    # OUTPUT:
    # reduced_derivative_assignments_dict: a dictionary containing an IBP-reduced set of derivative assignments
    # deleted: a list containing all removed derivative assignments
    # EXPLANATION:
    # scans through all derivative assignments and performs IBP reduction. it removes one operator for each 
    # IBP relation, which appears in one and only one IBP relation. in practice, it does this by removing 
    # any derivative assignments for which the maximum number of derivatives acting on any field is unique, and
    # for which there is not one fewer derivative acting on any field that occurs earlier in the specified ordering
    # of field types. it is worth noting that there is an element of conventionality to this prescription, and that 
    # other IBP reductions are possible that remove different sets of derivative assignments. 
    
    # generate derivative assignments and make copy of dictionary to use for deletion
    derivative_assignments_dict = generate_derivative_assignments(field_multiset)
    derivative_assignments_dict_reduced = deepcopy(derivative_assignments_dict)
    
    # declare empty list to store removed derivative assignments
    deleted = []
    
    #first_is_F = 'F' in field_multiset.keys()
    
    # loop through derivative assignments and remove from copy of dictionary 
    # those derivative assignments that meet the criteria for removal, appending them to 
    # the list of deleted items
    for outer_partition in derivative_assignments_dict.keys():
        #print('outer partition: ' + str(outer_partition))
        inner_partitions_dict = derivative_assignments_dict[outer_partition] 
        for inner_partition in inner_partitions_dict.keys():
            #print('inner partition: ' + str(inner_partition))
            full_partitions_list = inner_partitions_dict[inner_partition]
            full_partitions_list_reduced = derivative_assignments_dict_reduced[outer_partition][inner_partition]
            for derivative_assignment in full_partitions_list:
                #print(derivative_assignment)
                if IBP_remove(derivative_assignment):
                    #print('REMOVE: ' + str(derivative_assignment))
                    deleted.append(derivative_assignment)
                    #print(len(derivative_assignments_dict[outer_partition][inner_partition]))
                    full_partitions_list_reduced.remove(derivative_assignment) 
                    #print(len(derivative_assignments_dict[outer_partition][inner_partition]))
        
    return derivative_assignments_dict_reduced, deleted





In [ ]:
field_multiset = {’D’:2, ’F’:1, ’S’:1, ’V’: 2}

In [114]:
derivative_assignment = [[1, 2]]
print(IBP_remove(derivative_assignment))

False


In [146]:
field_multiset = {'D':6, 'F':3, 'S':1, 'V':2}

derivative_assignments = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments.keys():
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments[key1][key2]:
            print(value)


outer_partition: (6, 0, 0)
inner_partition: ((6, 0, 0), (0,), (0, 0))
[[6, 0, 0], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((5, 1, 0), (0,), (0, 0))
[[5, 1, 0], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((4, 2, 0), (0,), (0, 0))
[[4, 2, 0], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((4, 1, 1), (0,), (0, 0))
[[4, 1, 1], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((3, 3, 0), (0,), (0, 0))
[[3, 3, 0], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((3, 2, 1), (0,), (0, 0))
[[3, 2, 1], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((2, 2, 2), (0,), (0, 0))
[[2, 2, 2], [(0, 0)], [(0, 0), (0, 0)]]

outer_partition: (0, 6, 0)
inner_partition: ((0, 0, 0), (6,), (0, 0))
[[0, 0, 0], [(6, 0)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(0, 6)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(5, 1)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(1, 5)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(4, 2)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(2, 4)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(3, 3)], [(0, 0), (0, 0)]]

outer_partition: (0, 0, 6)
inner_partit

In [145]:
field_multiset = {'D':6, 'F':3}

derivative_assignments = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments.keys():
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments[key1][key2]:
            print(value)


outer_partition: (6,)
inner_partition: ((6, 0, 0),)
[[6, 0, 0]]
inner_partition: ((5, 1, 0),)
[[5, 1, 0]]
inner_partition: ((4, 2, 0),)
[[4, 2, 0]]
inner_partition: ((4, 1, 1),)
[[4, 1, 1]]
inner_partition: ((3, 3, 0),)
[[3, 3, 0]]
inner_partition: ((3, 2, 1),)
[[3, 2, 1]]
inner_partition: ((2, 2, 2),)
[[2, 2, 2]]


In [124]:
field_multiset = {'D':6, 'S':5}

derivative_assignments = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments.keys():
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments[key1][key2]:
            print(value)


outer_partition: (6,)
inner_partition: ((6, 0, 0, 0, 0),)
[[(6, 0), (0, 0), (0, 0), (0, 0), (0, 0)]]
[[(0, 6), (0, 0), (0, 0), (0, 0), (0, 0)]]
[[(5, 1), (0, 0), (0, 0), (0, 0), (0, 0)]]
[[(1, 5), (0, 0), (0, 0), (0, 0), (0, 0)]]
[[(4, 2), (0, 0), (0, 0), (0, 0), (0, 0)]]
[[(2, 4), (0, 0), (0, 0), (0, 0), (0, 0)]]
[[(3, 3), (0, 0), (0, 0), (0, 0), (0, 0)]]
inner_partition: ((5, 1, 0, 0, 0),)
[[(5, 0), (1, 0), (0, 0), (0, 0), (0, 0)]]
[[(5, 0), (0, 1), (0, 0), (0, 0), (0, 0)]]
[[(0, 5), (1, 0), (0, 0), (0, 0), (0, 0)]]
[[(0, 5), (0, 1), (0, 0), (0, 0), (0, 0)]]
[[(4, 1), (1, 0), (0, 0), (0, 0), (0, 0)]]
[[(4, 1), (0, 1), (0, 0), (0, 0), (0, 0)]]
[[(1, 4), (1, 0), (0, 0), (0, 0), (0, 0)]]
[[(1, 4), (0, 1), (0, 0), (0, 0), (0, 0)]]
[[(3, 2), (1, 0), (0, 0), (0, 0), (0, 0)]]
[[(3, 2), (0, 1), (0, 0), (0, 0), (0, 0)]]
[[(2, 3), (1, 0), (0, 0), (0, 0), (0, 0)]]
[[(2, 3), (0, 1), (0, 0), (0, 0), (0, 0)]]
inner_partition: ((4, 2, 0, 0, 0),)
[[(4, 0), (2, 0), (0, 0), (0, 0), (0, 0)]]
[[(4, 0),

In [125]:
field_multiset = {'D':3, 'F':3, 'S':1, 'V': 2}

derivative_assignments = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments.keys():
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments[key1][key2]:
            print(value)


outer_partition: (3, 0, 0)
inner_partition: ((3, 0, 0), (0,), (0, 0))
[[3, 0, 0], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((2, 1, 0), (0,), (0, 0))
[[2, 1, 0], [(0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((1, 1, 1), (0,), (0, 0))
[[1, 1, 1], [(0, 0)], [(0, 0), (0, 0)]]

outer_partition: (0, 3, 0)
inner_partition: ((0, 0, 0), (3,), (0, 0))
[[0, 0, 0], [(3, 0)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(0, 3)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(2, 1)], [(0, 0), (0, 0)]]
[[0, 0, 0], [(1, 2)], [(0, 0), (0, 0)]]

outer_partition: (0, 0, 3)
inner_partition: ((0, 0, 0), (0,), (3, 0))
[[0, 0, 0], [(0, 0)], [(3, 0), (0, 0)]]
[[0, 0, 0], [(0, 0)], [(0, 3), (0, 0)]]
[[0, 0, 0], [(0, 0)], [(2, 1), (0, 0)]]
[[0, 0, 0], [(0, 0)], [(1, 2), (0, 0)]]
inner_partition: ((0, 0, 0), (0,), (2, 1))
[[0, 0, 0], [(0, 0)], [(2, 0), (1, 0)]]
[[0, 0, 0], [(0, 0)], [(2, 0), (0, 1)]]
[[0, 0, 0], [(0, 0)], [(0, 2), (1, 0)]]
[[0, 0, 0], [(0, 0)], [(0, 2), (0, 1)]]
[[0, 0, 0], [(0, 0)], [(1, 1), (1, 0)]]
[[0, 0, 0], [(0, 0

In [621]:
field_multiset = {'D':4, 'F':4, 'S':1, 'V': 2, 'T':3}


derivative_assignments = generate_derivative_assignments(field_multiset)
derivative_assignments_IBP_reduced, deleted = IBP_reduce(field_multiset)

for key1 in derivative_assignments.keys():
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_IBP_reduced[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_IBP_reduced[key1][key2]:
            print(value)

print('')
print('deleted')
for i in range(len(deleted)):
    print(deleted[i])



outer_partition: (4, 0, 0, 0)
inner_partition: ((4, 0, 0, 0), (0,), (0, 0), (0, 0, 0))
inner_partition: ((3, 1, 0, 0), (0,), (0, 0), (0, 0, 0))
inner_partition: ((2, 2, 0, 0), (0,), (0, 0), (0, 0, 0))
[[2, 2, 0, 0], [(0, 0)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
inner_partition: ((2, 1, 1, 0), (0,), (0, 0), (0, 0, 0))
inner_partition: ((1, 1, 1, 1), (0,), (0, 0), (0, 0, 0))
[[1, 1, 1, 1], [(0, 0)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]

outer_partition: (0, 4, 0, 0)
inner_partition: ((0, 0, 0, 0), (4,), (0, 0), (0, 0, 0))
[[0, 0, 0, 0], [(2, 2)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]

outer_partition: (0, 0, 4, 0)
inner_partition: ((0, 0, 0, 0), (0,), (4, 0), (0, 0, 0))
[[0, 0, 0, 0], [(0, 0)], [(2, 2), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
inner_partition: ((0, 0, 0, 0), (0,), (3, 1), (0, 0, 0))
[[0, 0, 0, 0], [(0, 0)], [(1, 2), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
[[0, 0, 0, 0], [(0, 0)], [(1, 2), (0, 1)], [(0, 0), (0, 0), (0, 0)]]
inner_partition: ((0, 0, 0, 0), (0,),

In [626]:
field_multiset = {'D':3, 'V': 1}



derivative_assignments = generate_derivative_assignments(field_multiset)
#derivative_assignments_IBP_reduced, deleted = IBP_reduce(field_multiset)

for key1 in derivative_assignments.keys():
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments[key1][key2]:
            print(value)
'''
print('')
print('deleted')
for i in range(len(deleted)):
    print(deleted[i])
'''



outer_partition: (3,)
inner_partition: ((3,),)
[[(3, 0)]]
[[(0, 3)]]
[[(2, 1)]]
[[(1, 2)]]


"\nprint('')\nprint('deleted')\nfor i in range(len(deleted)):\n    print(deleted[i])\n"

In [627]:
derivative_assignments_IBP_reduced, deleted = IBP_reduce(field_multiset)

for key1 in derivative_assignments_IBP_reduced.keys():
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_IBP_reduced[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_IBP_reduced[key1][key2]:
            print(value)

print('')
print('deleted')
for i in range(len(deleted)):
    print(deleted[i])



outer_partition: (3,)
inner_partition: ((3,),)
[[(1, 2)]]

deleted
[[(3, 0)]]
[[(0, 3)]]
[[(2, 1)]]


## Generate All Contractions of Free Indices in Multiset with Derivative Assignments


To generate Lorentz contractions, we will use a special encoding of a derivative assignment that groups together derivatives and field objects that can be freely interchanged without changing the generated operator. Consider the following example

\begin{align}
field\_multiset = \{ D:12, F:4, V:2, T:1 \}
\end{align}

\begin{align}
derivative\_assignment = [[2,2,2,0], [(0,1),(0,1)], [(1,3)]]
\end{align}

Together these objects can be grouped as follows

\begin{align}
([D^{2}][F])([D^{2}][F])([D^{2}][F])([F])(\bar{\psi}[V][D]\psi)(\bar{\psi}[V][D]\psi)([D]\bar{\psi}[T][D^3]\psi)
\end{align}

Lorentz contractions are then between groups in squared brackets, where each group has a Lorentz "valence" associated with the number of interchangeable indices that can be contracted. To encode the grouping, we use nested lists, in this case

\begin{align}
[[D^2, F], [D^2, F], [D^2, F], [F], [V, D], [V, D], [D, T, D^3]]. 
\end{align}

\noindent Now we index the $14$ basic operators $D^2, F, D^2, F, D^2, F, F, V, D, V, D, D, T, D^3$ in order from 0, 1, 2, ..., 13. We also write a list of Lorentz ranks $[2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 3]$ associated with each such operator, which will be needed when generating a list of ways to fully Lorentz contract the term.  

To encode the set of contractions,

\begin{align}
(D_{\mu_1} D^{\mu_1} F^{\mu_2 \mu_3})(D_{\mu_4} D_{\mu_5} F_{\mu_2 \mu_3})(D^{\mu_4} D_{\mu_6} F^{\mu_5 \mu_7})(F_{\mu_7 \mu_8})(\bar{\psi}V^{\mu_6}D^{\mu_{10}}\psi)(\bar{\psi}V_{\mu_9}D_{\mu_{10}}\psi)(D_{\mu_{11}}\bar{\psi}T^{\mu_8 \mu_9}D^{\mu_{11}}D_{\mu_{12}}D^{\mu_{12}}\psi)
\end{align}

we would write 

\begin{align}
[(0,0), (1,3), (1,3), (2,4), (2,5), (4,7), (5,6), (6,12), (8,10), (9,12), (11,13), (13, 13)].
\end{align}

We should encode each contraction as a frozen set, since the order of the two integers does not matter, and each way of contracting a derivative assignment as a multiset or dictionary of frozen sets indicating the multiplicity of each contraction. We use frozen sets so they can be used as keys in the dictionary. 

Without changing the resulting operator, we can arbitrarily permute the ordered tuples of indices $(0,1) <-> (2,3) <-> (3,4)$ and also the ordered tuples $(7,8) <-> (9,10)$ since their respective operator groups are identical.



In [180]:
multiset1 = {frozenset((0,1)):2, frozenset((0,2)):3, frozenset((3,2)):5}
multiset2 = {frozenset((1,0)):2, frozenset((2,3)):5, frozenset((0,2)):3}
multiset1 == multiset2

True

In [165]:
field_multiset = { 'D':12, 'F':4, 'V':2, 'T':1 }
l = list(field_multiset.keys())
l.remove('D')
print(l)

['F', 'V', 'T']


In [766]:
def encode_lorentz_grouping(field_multiset, derivative_assignment):
    # check compatibility of field_multiset and derivative_assignment. 
    # first, check that number of non-derivative elements in field_multiset is equal to the number of sub-lists in 
    # derivative_assignment.
    #print('IN ENCODE_LORENTZ_GROUPINGS()')
    #print('field_multiset: ' + str(field_multiset))
    nonderivative_fields = list(field_multiset.keys())
    if 'D' in nonderivative_fields:
        nonderivative_fields.remove('D')
    if len(nonderivative_fields) != len(derivative_assignment):
        print('In encode_lorentz_grouping(). Arguments are incompatible.')
        
    lorentz_grouping = []
    num_field_types = len(derivative_assignment)
    for i in range(num_field_types):
        field_type = nonderivative_fields[i]
        derivative_dist = derivative_assignment[i]
        num_copies = len(derivative_dist)
        for j in range(num_copies):
            # for F-type fields
            if field_type == 'F':
                nD = derivative_dist[j]
                derivative_group = [nD*'D' for i in range(1) if nD*'D'] #a bit hacky. leaves list empty if nD is zero
                group = derivative_group + [field_type]
            # for bilinear fields
            else:
                two_tuple = derivative_dist[j]
                nD_left = two_tuple[0]
                nD_right = two_tuple[1]
                derivative_group_left = [nD_left*'D' for i in range(1) if nD_left*'D'] #a bit hacky. leaves list empty if nD_left is zero
                #print(derivative_group_left)
                derivative_group_right = [nD_right*'D' for i in range(1) if nD_right*'D']
                #print(derivative_group_right)
                #print([field_type])
                group = derivative_group_left + [field_type] + derivative_group_right
                
            lorentz_grouping.append(group)
    
    # group indices in nested list by group type, group copy
    index_grouping_by_type = []
    index = 0
    num_groups = len(lorentz_grouping)
    group_type = []
    for i in range(num_groups):
        #print('i: ' + str(i))
        group = lorentz_grouping[i]
        #print(group)
        num_items = len(group)
        
        group_indices = []
        for j in range(num_items):
            group_indices.append(index)
            #print(index)
            index += 1
        #print(group_indices)
        
        if i == 0:
            #print('i==0')
            group_type = [group_indices]
            #print('group_type: ' + str(group_type))
            if i == num_groups - 1:
                    #print('i == num_groups - 1')
                    index_grouping_by_type.append(group_type)    
        else:
            if group == lorentz_grouping[i-1]:
                #print('group == lorentz_grouping[i-1]')
                group_type.append(group_indices)
                #print('group_type: ' + str(group_type))
                if i == num_groups - 1:
                    #print('i == num_groups - 1')
                    index_grouping_by_type.append(group_type)
            else:
                index_grouping_by_type.append(group_type)
                group_type = [group_indices]
                if i == num_groups - 1:
                    #print('i == num_groups - 1')
                    index_grouping_by_type.append(group_type)
        #print(group_type)
        
    
    return lorentz_grouping, index_grouping_by_type 


In [767]:
field_multiset = {'D':2, 'F':1}
derivative_assignment = [[10]]

lorentz_grouping, index_grouping_by_type  = encode_lorentz_grouping(field_multiset, derivative_assignment)
lorentz_grouping, index_grouping_by_type 

([['DDDDDDDDDD', 'F']], [[[0, 1]]])

In [749]:
field_multiset = {'D':10, 'F':1}
derivative_assignment = [[10]]

lorentz_grouping, index_grouping_by_type  = encode_lorentz_grouping(field_multiset, derivative_assignment)
lorentz_grouping, index_grouping_by_type 

IN ENCODE_LORENTZ_GROUPINGS()


([['DDDDDDDDDD', 'F']], [[[0, 1]]])

In [750]:
field_multiset = {'F':3}
derivative_assignment = [[0, 0, 0]]

lorentz_grouping, index_grouping_by_type  = encode_lorentz_grouping(field_multiset, derivative_assignment)
lorentz_grouping, index_grouping_by_type 

IN ENCODE_LORENTZ_GROUPINGS()


([['F'], ['F'], ['F']], [[[0], [1], [2]]])

In [751]:
field_multiset = {'D': 14,'F': 4, 'V': 3, 'T': 1}
derivative_assignment = [[2,2,2,0],[(0,1),(0,1),(1,1)],[(1,3)]]

lorentz_grouping, index_grouping_by_type  = encode_lorentz_grouping(field_multiset, derivative_assignment)
lorentz_grouping, index_grouping_by_type 

IN ENCODE_LORENTZ_GROUPINGS()


([['DD', 'F'],
  ['DD', 'F'],
  ['DD', 'F'],
  ['F'],
  ['V', 'D'],
  ['V', 'D'],
  ['D', 'V', 'D'],
  ['D', 'T', 'DDD']],
 [[[0, 1], [2, 3], [4, 5]],
  [[6]],
  [[7, 8], [9, 10]],
  [[11, 12, 13]],
  [[14, 15, 16]]])

In [789]:
def list_lorentz_ranks(lorentz_grouping_flattened):
    ranks_list = []
    lorRanks = {'D': 1, 'F': 2, 'S': 0, 'V': 1, 'T': 2, 'Vp': 1, 'Sp': 0}
    n_items = len(lorentz_grouping_flattened)
    for i in range(n_items):
        group = lorentz_grouping_flattened[i]
        group_lorRank = 0
        if group[0]=='D':
            for j in range(len(group)):
                group_lorRank += lorRanks[group[j]]
        else:
            group_lorRank += lorRanks[group]
        ranks_list.append(group_lorRank)    
    return ranks_list

In [790]:
field_multiset = {'D': 14,'F': 4, 'V': 3, 'T': 1}
derivative_assignment = [[2,2,2,0],[(0,1),(0,1),(1,1)],[(1,3)]]
lorentz_grouping,_ = encode_lorentz_grouping(field_multiset, derivative_assignment)
lorentz_grouping_flattened = [x for group in lorentz_grouping for x in group]
print(lorentz_grouping_flattened)
list_lorentz_ranks(lorentz_grouping_flattened)

['DD', 'F', 'DD', 'F', 'DD', 'F', 'F', 'V', 'D', 'V', 'D', 'D', 'V', 'D', 'D', 'T', 'DDD']


[2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3]

In [791]:
field_multiset = {'D': 18,'F': 4, 'V': 3, 'T': 1, 'Vp': 2, 'Sp': 1}
derivative_assignment = [[2,2,2,0],[(0,1),(0,1),(1,1)],[(1,3)],[(1,1)],[(0,2)]]
lorentz_grouping,_ = encode_lorentz_grouping(field_multiset, derivative_assignment)
lorentz_grouping_flattened = [x for group in lorentz_grouping for x in group]
print(lorentz_grouping_flattened)
list_lorentz_ranks(lorentz_grouping_flattened)

['DD', 'F', 'DD', 'F', 'DD', 'F', 'F', 'V', 'D', 'V', 'D', 'D', 'V', 'D', 'D', 'T', 'DDD', 'D', 'Vp', 'D', 'Sp', 'DD']


[2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 0, 2]

In [517]:
# write code and test for {F: 3}, {F: 4}, etc with no derivatives. 
from multiset import FrozenMultiset

def generate_lorentz_contractions_from_ranks(lorentz_ranks_list, i0):
    # base case: if the total number of free Lorentz indices is 0 or 1, return empty list of contraction sets 
    if sum(lorentz_ranks_list) < 2:
        contraction_multisets_list = [{}]
        return contraction_multisets_list
    
    # find first field of non-zero rank for first index of contraction
    N_groups = len(lorentz_ranks_list)
    for i in range(N_groups):
        if lorentz_ranks_list[i] >= 1:
            i_start = i
            lorentz_ranks_list[i_start] -= 1
            #print('i_start: ' + str(i_start))
            break
            
    if i0 <= i_start:
        i0 = i_start
    
    contraction_multisets_list = []
    # find all ways of contracting first non-zero lorentz rank field with other fields or itself.
    for i in range(i0, N_groups):
        #print(i)
        if lorentz_ranks_list[i] >= 1:
            contraction = (i_start, i)
            
            lorentz_ranks_list_old = lorentz_ranks_list.copy()
            
            #decrement lorentzRanks_list_old 
            lorentz_ranks_list_old[i] -= 1
            
            #if i_start of decremented list is the same as the non-decremented list, only include contractions
            #where second index is greater than or equal to i
            if lorentz_ranks_list_old[i_start] > 0:
                contraction_multisets_list_old = generate_lorentz_contractions_from_ranks(lorentz_ranks_list_old, i)
            else: 
                contraction_multisets_list_old = generate_lorentz_contractions_from_ranks(lorentz_ranks_list_old, 0)
            
            for contraction_multiset_old in contraction_multisets_list_old:
                contraction_multiset = contraction_multiset_old.copy()
                if contraction in contraction_multiset.keys():
                    contraction_multiset[contraction] += 1
                else: 
                    contraction_multiset[contraction] = 1
                contraction_multisets_list.append(contraction_multiset) 
                
    return contraction_multisets_list




In [496]:
lorentz_ranks_list = [2,2,2,2,3,3,1,3]
generate_lorentz_contractions_from_ranks(lorentz_ranks_list, 0)

[{(7, 7): 1,
  (6, 7): 1,
  (5, 5): 1,
  (4, 5): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(7, 7): 1,
  (5, 7): 1,
  (5, 6): 1,
  (4, 5): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(6, 7): 1,
  (5, 7): 2,
  (4, 5): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(7, 7): 1,
  (5, 7): 1,
  (5, 5): 1,
  (4, 6): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(5, 7): 3, (4, 6): 1, (4, 4): 1, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1},
 {(7, 7): 1,
  (5, 6): 1,
  (5, 5): 1,
  (4, 7): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(6, 7): 1,
  (5, 7): 1,
  (5, 5): 1,
  (4, 7): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(5, 7): 2,
  (5, 6): 1,
  (4, 7): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(7, 7): 1, (6, 7): 1, (4, 5): 3, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1},
 {(7, 7): 1,
  (5, 7): 1,
  (4, 6): 1,
 

In [692]:

def generate_lorentz_contractions(field_multiset, derivative_assignment):
    #print('IN GENERATE_LORENTZ_CONTRACTIONS()')
    lorentz_grouping,_ = encode_lorentz_grouping(field_multiset, derivative_assignment)
    lorentz_grouping_flattened = [x for group in lorentz_grouping for x in group]
    #print('lorentz_grouping_flattened: ' + str(lorentz_grouping_flattened))
    lorentz_ranks_list = list_lorentz_ranks(lorentz_grouping_flattened)
    #print('lorentz_ranks_list: ' + str(lorentz_ranks_list))
    contraction_multisets_list = generate_lorentz_contractions_from_ranks(lorentz_ranks_list, 0)
    return contraction_multisets_list


In [498]:

field_multiset = {'D': 8,'F': 3, 'V': 2}
derivative_assignment = [[2,2,2],[(0,1),(0,1)]]
generate_lorentz_contractions(field_multiset, derivative_assignment)


IN ENCODE_LORENTZ_GROUPINGS()
field_multiset: {'D': 8, 'F': 3, 'V': 2}


[{(8, 9): 1,
  (8, 8): 1,
  (6, 7): 1,
  (6, 6): 1,
  (5, 5): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(8, 9): 1,
  (7, 8): 1,
  (6, 8): 1,
  (6, 6): 1,
  (5, 5): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(8, 8): 1,
  (7, 9): 1,
  (6, 8): 1,
  (6, 6): 1,
  (5, 5): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(8, 8): 1,
  (7, 8): 1,
  (6, 9): 1,
  (6, 6): 1,
  (5, 5): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(8, 9): 1,
  (6, 8): 2,
  (6, 7): 1,
  (5, 5): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(8, 8): 1,
  (6, 9): 1,
  (6, 8): 1,
  (6, 7): 1,
  (5, 5): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(7, 9): 1,
  (6, 8): 3,
  (5, 5): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(7, 8): 1,
  (6, 9): 1,
  (6, 8): 2,
  (5, 5): 1,
  (4, 4): 1,
  (3, 3): 1,
  (2, 2): 1,
  (1, 1): 1,
  (0, 0): 1},
 {(8

In [499]:
field_multiset = {'D': 8,'F': 3, 'V': 2}
derivative_assignment = [[2,2,2],[(0,1),(0,1)]]

lorentz_grouping, index_grouping_by_type  = encode_lorentz_grouping(field_multiset, derivative_assignment)
print(lorentz_grouping)
print(index_grouping_by_type)
lorentz_grouping_flattened = [x for group in lorentz_grouping for x in group]
print(lorentz_grouping_flattened)
lorentz_ranks_list = list_lorentz_ranks(lorentz_grouping_flattened)
print(lorentz_ranks_list)
#generate_lorentz_contractions_from_ranks(lorentz_ranks_list, 0) #takes too long


IN ENCODE_LORENTZ_GROUPINGS()
field_multiset: {'D': 8, 'F': 3, 'V': 2}
[['DD', 'F'], ['DD', 'F'], ['DD', 'F'], ['V', 'D'], ['V', 'D']]
[[[0, 1], [2, 3], [4, 5]]]
['DD', 'F', 'DD', 'F', 'DD', 'F', 'V', 'D', 'V', 'D']
[2, 2, 2, 2, 2, 2, 3, 1, 3, 1]


In [500]:
field_multiset = {'D': 14,'F': 4, 'V': 3, 'T': 1}
derivative_assignment = [[2,2,2,0],[(0,1),(0,1), (1,1)],[(1,3)]]

lorentz_grouping, index_grouping_by_type  = encode_lorentz_grouping(field_multiset, derivative_assignment)
lorentz_grouping, index_grouping_by_type 

IN ENCODE_LORENTZ_GROUPINGS()
field_multiset: {'D': 14, 'F': 4, 'V': 3, 'T': 1}


([['DD', 'F'],
  ['DD', 'F'],
  ['DD', 'F'],
  ['F'],
  ['V', 'D'],
  ['V', 'D'],
  ['D', 'V', 'D'],
  ['D', 'T', 'DDD']],
 [[[0, 1], [2, 3], [4, 5]],
  [[6]],
  [[7, 8], [9, 10]],
  [[11, 12, 13]],
  [[14, 15, 16]]])

In [771]:
import itertools

def flatten(index_grouping_by_type):
    return [x for group_type in index_grouping_by_type for copy in group_type for x in copy]

def replace_indices(contraction_tuple, index_map):
    #print('IN REPLACE_INDICES()')
    #print('index_map: ' + str(index_map))
    #print('contraction_tuple: ' + str(contraction_tuple))
    contraction_tuple_new = tuple(sorted((index_map[contraction_tuple[0]], index_map[contraction_tuple[1]])))
    return contraction_tuple_new
    
def substitute_permuted_indices(lorentz_contraction, index_map):
    #print('IN SUBSTITUTE_PERMUTED_INDICES()')
    lorentz_contraction_new = {}
    for contraction_tuple in lorentz_contraction.keys():
        #print('contraction_tuple: ' + str(contraction_tuple))
        contraction_tuple_new = replace_indices(contraction_tuple, index_map)
        #print('contraction_tuple_new: ' + str(contraction_tuple_new))
        lorentz_contraction_new[contraction_tuple_new] = lorentz_contraction[contraction_tuple]
    return lorentz_contraction_new

def permuted_contractions(lorentz_contraction, index_grouping_by_type):
    #print('IN PERMUTED_CONTRACTIONS()')
    # generates all permutations of lorentz_contraction equivalent under index_grouping_by_type. for each group
    # type with more than one copy - i.e., for each sublist with more than one subsublist - 
    
    '''
    # for each group type, generate a list of all permutations of indices associated with copies of that group type.
    # assemble a list of these lists, called permutations_by_type
    permutations_by_type = []
    for group_type in index_grouping_by_type:
        permuted_contractions = []
        permutations_list = list(itertools.permutations(group_type))
        permutations_by_type.append(permutations_list)
    '''
    
    # generate all possible sequences of permutations, where each sequence consists of one permutation for each group
    # type. 
    #print('index_grouping_by_type: ' + str(index_grouping_by_type))
    N_group_types = len(index_grouping_by_type)
    #print('N_group_types')
    full_permutations_list = [[]]
    counter = 0
    while counter < N_group_types:
        #print('counter: ' + str(counter))
        full_permutations_list_old = full_permutations_list.copy()
        full_permutations_list = []
        group_type = index_grouping_by_type[counter]
        permutations_list = list(itertools.permutations(group_type))
        #print(permutations_list)
        for full_permutation in full_permutations_list_old:
            #print('full_permutation: ' + str(full_permutation))
            for permutation in permutations_list:
                #print('permutation: ' + str(permutation))
                full_permutation_new = full_permutation + [list(permutation)]
                full_permutations_list.append(full_permutation_new)
        #print(full_permutations_list)
        counter += 1
    #print('full_permutations_list: ' + str(full_permutations_list))
                
    # for each possible sequence of permutations, generate an index map from original indices to permuted ones
    permuted_contractions_list = []
    index_grouping_by_type_flattened = flatten(index_grouping_by_type)
    for full_permutation in full_permutations_list:
        #print('full_permutation: ' + str(full_permutation))
        index_grouping_by_type_permuted_flattened = flatten(full_permutation)
        #print('index_grouping_by_type_permuted_flattened: ' + str(index_grouping_by_type_permuted_flattened))
        index_map = {i: index_grouping_by_type_permuted_flattened[i] for i in index_grouping_by_type_flattened}
        #print('index_map: ' + str(index_map))
        lorentz_contraction_permuted = substitute_permuted_indices(lorentz_contraction, index_map) 
        permuted_contractions_list.append(lorentz_contraction_permuted)
        
    return permuted_contractions_list





In [772]:
field_multiset = {'D':6, 'F':3, 'V':1}
derivative_assignment = [[2, 2, 2], [(0,0)]]
lorentz_grouping, index_grouping_by_type = encode_lorentz_grouping(field_multiset, derivative_assignment)
lorentz_contraction = {(4, 6): 2, (2, 5): 1, (2, 3): 1, (1, 6): 1, (0, 3): 1, (0, 1): 1}

permuted_contractions(lorentz_contraction, index_grouping_by_type)

[{(4, 6): 2, (2, 5): 1, (2, 3): 1, (1, 6): 1, (0, 3): 1, (0, 1): 1},
 {(2, 6): 2, (3, 4): 1, (4, 5): 1, (1, 6): 1, (0, 5): 1, (0, 1): 1},
 {(4, 6): 2, (0, 5): 1, (0, 1): 1, (3, 6): 1, (1, 2): 1, (2, 3): 1},
 {(0, 6): 2, (1, 4): 1, (4, 5): 1, (3, 6): 1, (2, 5): 1, (2, 3): 1},
 {(2, 6): 2, (0, 3): 1, (0, 1): 1, (5, 6): 1, (1, 4): 1, (4, 5): 1},
 {(0, 6): 2, (1, 2): 1, (2, 3): 1, (5, 6): 1, (3, 4): 1, (4, 5): 1}]

In [569]:
field_multiset = {'D':6, 'F':3, 'V':1}
derivative_assignment = [[2, 2, 2], [(0,0)]]
lorentz_grouping, index_grouping_by_type = encode_lorentz_grouping(field_multiset, derivative_assignment)
print(lorentz_grouping)
print('')
print(index_grouping_by_type)

[['DD', 'F'], ['DD', 'F'], ['DD', 'F'], ['V']]

[[[0, 1], [2, 3], [4, 5]], [[6]]]


In [570]:
lorentz_grouping_flattened = [x for group in lorentz_grouping for x in group]
print(lorentz_grouping_flattened)
lorentz_ranks_list = list_lorentz_ranks(lorentz_grouping_flattened)
print(lorentz_ranks_list)

['DD', 'F', 'DD', 'F', 'DD', 'F', 'V']
[2, 2, 2, 2, 2, 2, 3]


In [571]:
#generate_lorentz_contractions(lorentz_ranks_list,0)

In [572]:
field_multiset = {'D':6, 'F':3, 'V':1}
derivative_assignment = [[2, 2, 2], [(0,0)]]

contraction_multisets_list = generate_lorentz_contractions(field_multiset, derivative_assignment)
print(contraction_multisets_list)

[{(6, 6): 1, (5, 5): 1, (4, 4): 1, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(5, 6): 2, (4, 4): 1, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(6, 6): 1, (4, 5): 2, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(5, 6): 1, (4, 6): 1, (4, 5): 1, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(5, 5): 1, (4, 6): 2, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(5, 6): 1, (4, 6): 2, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(6, 6): 1, (5, 5): 1, (3, 4): 2, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(5, 6): 2, (3, 4): 2, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(6, 6): 1, (4, 5): 1, (3, 5): 1, (3, 4): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(5, 6): 1, (4, 6): 1, (3, 5): 1, (3, 4): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(5, 6): 1, (4, 5): 1, (3, 6): 1, (3, 4): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(5, 5): 1, (4, 6): 1, (3, 6): 1, (3, 4): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(5, 6): 1, (4, 6): 1, (3, 6): 1, (3, 4): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}, {(6, 6): 1, (4, 4): 1, (3, 5): 2, (2, 2): 1, (1, 

In [573]:
print(len(contraction_multisets_list))

4109


In [774]:
def lorentz_contractions_equiv(lorentz_contraction1, lorentz_contraction2, field_multiset, derivative_assignment):
    #print('IN LORENTZ_CONTRACTIONS_EQUIV()')
    #print('field_multiset: ' + str(field_multiset))
    #print('derivative_assignment: ' + str(derivative_assignment))
    _, index_grouping_by_type = encode_lorentz_grouping(field_multiset, derivative_assignment)
    #print('index_grouping_by_type: ' + str(index_grouping_by_type))
    if lorentz_contraction2 in permuted_contractions(lorentz_contraction1, index_grouping_by_type):
        return True        
    else:                
        return False

def remove_repeat_lorentz_contractions(field_multiset, derivative_assignment):
    # THIS IS THE MAIN FUNCTION OF THIS SECTION
    #print('IN REMOVE_REPEAT_LORENTZ_CONTRACTIONS()')
    contraction_multisets_list = generate_lorentz_contractions(field_multiset, derivative_assignment)
    N_contractions = len(contraction_multisets_list)
    # start list of unique elements of contraction_multisets_list with first element
    contraction_multisets_list_unique = [contraction_multisets_list[0]] 
    #print('N_contractions: ' + str(N_contractions))
    for i in range(1,N_contractions):
        #print('i: ' + str(i))
        contraction_multiset = contraction_multisets_list[i]
        N_unique = len(contraction_multisets_list_unique)
        #print('N_unique: ' + str(N_unique))
        
        def already_in_list():
            for j in range(N_unique):
                contraction_multiset_unique = contraction_multisets_list_unique[j]
                equiv = lorentz_contractions_equiv(contraction_multiset, contraction_multiset_unique, field_multiset, derivative_assignment)
                if equiv:
                    return True
            return False
        
        if already_in_list():
            continue
        else:
            contraction_multisets_list_unique.append(contraction_multiset)
    return contraction_multisets_list_unique
            
            


In [775]:
field_multiset = {'D':6, 'F':3, 'V':1}
derivative_assignment = [[2, 2, 2], [(0,0)]]

lorentz_contraction1 = {(4, 6): 2, (2, 5): 1, (2, 3): 1, (1, 6): 1, (0, 3): 1, (0, 1): 1}
#lorentz_contraction2 = {(2, 6): 2, (3, 4): 1, (4, 5): 1, (1, 6): 1, (0, 5): 1, (0, 1): 1}
lorentz_contraction2 = {(0, 6): 2, (1, 2): 1, (2, 3): 1, (5, 6): 1, (3, 4): 1, (4, 5): 1}

lorentz_contractions_equiv(lorentz_contraction1, lorentz_contraction2, field_multiset, derivative_assignment)

True

In [786]:
'''
field_multiset = {'D':6, 'F':3, 'V':1}
derivative_assignment = [[2, 2, 2], [(0,0)]]

lorentz_contractions_unique = remove_repeat_lorentz_contractions(field_multiset, derivative_assignment)
lorentz_contractions_unique
'''

"\nfield_multiset = {'D':6, 'F':3, 'V':1}\nderivative_assignment = [[2, 2, 2], [(0,0)]]\n\nlorentz_contractions_unique = remove_repeat_lorentz_contractions(field_multiset, derivative_assignment)\nlorentz_contractions_unique\n"

In [584]:
field_multiset = {'F':6}
derivative_assignment = [[0, 0, 0, 0, 0, 0]]

lorentz_grouping, index_groupings_by_type = encode_lorentz_grouping(field_multiset, derivative_assignment)

print(lorentz_grouping)
print(index_groupings_by_type) # should be [[[0],[1],[2]]]

lorentz_contractions_unique = remove_repeat_lorentz_contractions(field_multiset, derivative_assignment)
lorentz_contractions_unique

[['F'], ['F'], ['F'], ['F'], ['F'], ['F']]
[[[0], [1], [2], [3], [4], [5]]]
IN REMOVE_REPEAT_LORENTZ_CONTRACTIONS()
N_contractions: 388
i: 1
N_unique: 1
i: 2
N_unique: 2
i: 3
N_unique: 2
i: 4
N_unique: 3
i: 5
N_unique: 3
i: 6
N_unique: 3
i: 7
N_unique: 4
i: 8
N_unique: 4
i: 9
N_unique: 5
i: 10
N_unique: 5
i: 11
N_unique: 5
i: 12
N_unique: 5
i: 13
N_unique: 5
i: 14
N_unique: 5
i: 15
N_unique: 5
i: 16
N_unique: 5
i: 17
N_unique: 5
i: 18
N_unique: 5
i: 19
N_unique: 5
i: 20
N_unique: 5
i: 21
N_unique: 6
i: 22
N_unique: 6
i: 23
N_unique: 6
i: 24
N_unique: 6
i: 25
N_unique: 6
i: 26
N_unique: 7
i: 27
N_unique: 7
i: 28
N_unique: 7
i: 29
N_unique: 7
i: 30
N_unique: 7
i: 31
N_unique: 7
i: 32
N_unique: 7
i: 33
N_unique: 7
i: 34
N_unique: 7
i: 35
N_unique: 7
i: 36
N_unique: 7
i: 37
N_unique: 7
i: 38
N_unique: 7
i: 39
N_unique: 7
i: 40
N_unique: 7
i: 41
N_unique: 7
i: 42
N_unique: 7
i: 43
N_unique: 7
i: 44
N_unique: 7
i: 45
N_unique: 7
i: 46
N_unique: 7
i: 47
N_unique: 7
i: 48
N_unique: 7
i: 49
N_u

[{(5, 5): 1, (4, 4): 1, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1},
 {(4, 5): 2, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1},
 {(4, 5): 1, (3, 5): 1, (3, 4): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1},
 {(4, 5): 2, (2, 3): 2, (1, 1): 1, (0, 0): 1},
 {(4, 5): 1, (3, 5): 1, (2, 4): 1, (2, 3): 1, (1, 1): 1, (0, 0): 1},
 {(4, 5): 1, (3, 5): 1, (3, 4): 1, (1, 2): 2, (0, 0): 1},
 {(4, 5): 1, (3, 5): 1, (2, 4): 1, (1, 3): 1, (1, 2): 1, (0, 0): 1},
 {(4, 5): 2, (2, 3): 2, (0, 1): 2},
 {(4, 5): 1, (3, 5): 1, (2, 4): 1, (2, 3): 1, (0, 1): 2},
 {(4, 5): 1, (3, 5): 1, (3, 4): 1, (1, 2): 1, (0, 2): 1, (0, 1): 1},
 {(4, 5): 1, (3, 5): 1, (2, 4): 1, (1, 3): 1, (0, 2): 1, (0, 1): 1}]

In [599]:
field_multiset = {'F':6}
derivative_assignment = [[0, 0, 0, 0, 0, 0]]

lorentz_grouping, index_groupings_by_type = encode_lorentz_grouping(field_multiset, derivative_assignment)

print(lorentz_grouping)
print(index_groupings_by_type) # should be [[[0],[1],[2]]]

lorentz_contractions_unique = remove_repeat_lorentz_contractions(field_multiset, derivative_assignment)
lorentz_contractions_unique

[['F'], ['F'], ['F'], ['F'], ['F'], ['F']]
[[[0], [1], [2], [3], [4], [5]]]
IN REMOVE_REPEAT_LORENTZ_CONTRACTIONS()
N_contractions: 388
i: 1
N_unique: 1
i: 2
N_unique: 2
i: 3
N_unique: 2
i: 4
N_unique: 3
i: 5
N_unique: 3
i: 6
N_unique: 3
i: 7
N_unique: 4
i: 8
N_unique: 4
i: 9
N_unique: 5
i: 10
N_unique: 5
i: 11
N_unique: 5
i: 12
N_unique: 5
i: 13
N_unique: 5
i: 14
N_unique: 5
i: 15
N_unique: 5
i: 16
N_unique: 5
i: 17
N_unique: 5
i: 18
N_unique: 5
i: 19
N_unique: 5
i: 20
N_unique: 5
i: 21
N_unique: 6
i: 22
N_unique: 6
i: 23
N_unique: 6
i: 24
N_unique: 6
i: 25
N_unique: 6
i: 26
N_unique: 7
i: 27
N_unique: 7
i: 28
N_unique: 7
i: 29
N_unique: 7
i: 30
N_unique: 7
i: 31
N_unique: 7
i: 32
N_unique: 7
i: 33
N_unique: 7
i: 34
N_unique: 7
i: 35
N_unique: 7
i: 36
N_unique: 7
i: 37
N_unique: 7
i: 38
N_unique: 7
i: 39
N_unique: 7
i: 40
N_unique: 7
i: 41
N_unique: 7
i: 42
N_unique: 7
i: 43
N_unique: 7
i: 44
N_unique: 7
i: 45
N_unique: 7
i: 46
N_unique: 7
i: 47
N_unique: 7
i: 48
N_unique: 7
i: 49
N_u

[{(5, 5): 1, (4, 4): 1, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1},
 {(4, 5): 2, (3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1},
 {(4, 5): 1, (3, 5): 1, (3, 4): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1},
 {(4, 5): 2, (2, 3): 2, (1, 1): 1, (0, 0): 1},
 {(4, 5): 1, (3, 5): 1, (2, 4): 1, (2, 3): 1, (1, 1): 1, (0, 0): 1},
 {(4, 5): 1, (3, 5): 1, (3, 4): 1, (1, 2): 2, (0, 0): 1},
 {(4, 5): 1, (3, 5): 1, (2, 4): 1, (1, 3): 1, (1, 2): 1, (0, 0): 1},
 {(4, 5): 2, (2, 3): 2, (0, 1): 2},
 {(4, 5): 1, (3, 5): 1, (2, 4): 1, (2, 3): 1, (0, 1): 2},
 {(4, 5): 1, (3, 5): 1, (3, 4): 1, (1, 2): 1, (0, 2): 1, (0, 1): 1},
 {(4, 5): 1, (3, 5): 1, (2, 4): 1, (1, 3): 1, (0, 2): 1, (0, 1): 1}]

## Pull Together Pipeline: Generate All Lorentz Contractions from Mass Dimension

In [674]:
from frozendict import frozendict

def generate_operators(massDim, IBP=True, EOM=True, C=True, P=True, T=True):
    '''
    operator_dict[massDim][field_multiset][derivative_assigment] = contraction_multisets_list
    '''
    field_symbols = ['D', 'F', 'S', 'V', 'T', 'Vp', 'Sp']
    field_massDims = [1, 2, 3, 3, 3 ,3 ,3]
    field_lorentzRanks = [1, 2, 0, 1, 2, 1, 0]
    
    field_multisets_dict = generate_field_multisets(massDim, field_symbols, field_massDims, field_lorentzRanks)
    field_multiset_list = field_multisets_dict[massDim]
    print('In generate_operators(), field multisets generated')
    
    operator_dict = {}
    IBP_deleted_dict = {}
    for field_multiset in field_multiset_list:
        # for each field multiset, generate all derivative assignments
        print('field_multiset: ' + str(field_multiset))

        operator_dict_field_multiset = {}
        if IBP:
            derivative_assignments_dict, IBP_deleted = IBP_reduce(field_multiset)
            IBP_deleted_dict[frozendict(field_multiset)] = IBP_deleted
        else: 
            derivative_assignments_dict = generate_derivative_assignments(field_multiset)
            
        for outer_partition in derivative_assignments_dict.keys():
            for inner_partition in derivative_assignments_dict[outer_partition].keys(): 
                for derivative_assignment in derivative_assignments_dict[outer_partition][inner_partition]:
                    def make_hashable(derivative_assignment):
                        return tuple(tuple(sublist) for sublist in derivative_assignment)
                    print('derivative_assignment: ' + str(make_hashable(derivative_assignment)))
                    lorentz_contractions_unique = remove_repeat_lorentz_contractions(field_multiset, derivative_assignment)
                    operator_dict_field_multiset[make_hashable(derivative_assignment)] = lorentz_contractions_unique
        operator_dict[frozendict(field_multiset)] = operator_dict_field_multiset
    
    return operator_dict, IBP_deleted_dict

In [784]:
def display_generate_operators(massDim, IBP=True, EOM=True, C=True, P=True, T=True):
    # displays contents of operator_dict produced in generate_operators()
    operator_dict, IBP_deleted_dict = generate_operators(massDim, IBP, EOM, C, P, T)
    for field_multiset in operator_dict.keys():
        print('')
        print('')
        print('CONSTITUENT FIELDS: ')
        print(dict(field_multiset))
        for derivative_assignment in operator_dict[field_multiset].keys():
            print('')
            print('DERIVATIVE ASSIGNMENT: ')
            print(derivative_assignment)
            print('LORENTZ INDEX GROUPING: ')
            lorentz_grouping, index_groupings_by_type = encode_lorentz_grouping(field_multiset, derivative_assignment)
            print(lorentz_grouping)
            print(index_groupings_by_type)
            for lorentz_contraction_multiset in operator_dict[field_multiset][derivative_assignment]:
                print('LORENTZ CONTRACTIONS: ')
                print(lorentz_contraction_multiset)     
    
    if IBP:
        print('')
        print('DERIVATIVE ASSIGNMENTS REMOVED BASED ON IBP:')
        for field_multiset in IBP_deleted_dict.keys():
            print(field_multiset)
            for derivative_assignment in IBP_deleted_dict[field_multiset]:
                print(derivative_assignment)
    return operator_dict

In [793]:
massDim = 5
display_generate_operators(massDim, IBP=False)

In generate_operators(), field multisets generated
field_multiset: {'F': 1, 'S': 1}
derivative_assignment: ((0,), ((0, 0),))
field_multiset: {'F': 1, 'T': 1}
derivative_assignment: ((0,), ((0, 0),))
field_multiset: {'F': 1, 'Sp': 1}
derivative_assignment: ((0,), ((0, 0),))
field_multiset: {'D': 2, 'S': 1}
derivative_assignment: (((2, 0),),)
derivative_assignment: (((0, 2),),)
derivative_assignment: (((1, 1),),)
field_multiset: {'D': 2, 'T': 1}
derivative_assignment: (((2, 0),),)
derivative_assignment: (((0, 2),),)
derivative_assignment: (((1, 1),),)
field_multiset: {'D': 2, 'Sp': 1}
derivative_assignment: (((2, 0),),)
derivative_assignment: (((0, 2),),)
derivative_assignment: (((1, 1),),)


CONSTITUENT FIELDS: 
{'F': 1, 'S': 1}

DERIVATIVE ASSIGNMENT: 
((0,), ((0, 0),))
LORENTZ INDEX GROUPING: 
[['F'], ['S']]
[[[0]], [[1]]]
LORENTZ CONTRACTIONS: 
{(0, 0): 1}


CONSTITUENT FIELDS: 
{'F': 1, 'T': 1}

DERIVATIVE ASSIGNMENT: 
((0,), ((0, 0),))
LORENTZ INDEX GROUPING: 
[['F'], ['T']]
[[[0]]

{<frozendict {'F': 1, 'S': 1}>: {((0,), ((0, 0),)): [{(0, 0): 1}]},
 <frozendict {'F': 1, 'T': 1}>: {((0,), ((0, 0),)): [{(1, 1): 1, (0, 0): 1},
   {(0, 1): 2}]},
 <frozendict {'F': 1, 'Sp': 1}>: {((0,), ((0, 0),)): [{(0, 0): 1}]},
 <frozendict {'D': 2, 'S': 1}>: {(((2, 0),),): [{(0, 0): 1}],
  (((0, 2),),): [{(1, 1): 1}],
  (((1, 1),),): [{(0, 2): 1}]},
 <frozendict {'D': 2, 'T': 1}>: {(((2, 0),),): [{(1, 1): 1, (0, 0): 1},
   {(0, 1): 2}],
  (((0, 2),),): [{(1, 1): 1, (0, 0): 1}, {(0, 1): 2}],
  (((1, 1),),): [{(1, 2): 1, (0, 1): 1}, {(1, 1): 1, (0, 2): 1}]},
 <frozendict {'D': 2, 'Sp': 1}>: {(((2, 0),),): [{(0, 0): 1}],
  (((0, 2),),): [{(1, 1): 1}],
  (((1, 1),),): [{(0, 2): 1}]}}

In [794]:
massDim = 11
display_generate_operators(massDim, IBP=False)

In generate_operators(), field multisets generated
field_multiset: {'F': 1, 'S': 3}
derivative_assignment: ((0,), ((0, 0), (0, 0), (0, 0)))
field_multiset: {'F': 1, 'S': 2, 'T': 1}
derivative_assignment: ((0,), ((0, 0), (0, 0)), ((0, 0),))
field_multiset: {'F': 1, 'S': 2, 'Sp': 1}
derivative_assignment: ((0,), ((0, 0), (0, 0)), ((0, 0),))
field_multiset: {'F': 1, 'S': 1, 'V': 2}
derivative_assignment: ((0,), ((0, 0),), ((0, 0), (0, 0)))
field_multiset: {'F': 1, 'S': 1, 'V': 1, 'Vp': 1}
derivative_assignment: ((0,), ((0, 0),), ((0, 0),), ((0, 0),))
field_multiset: {'F': 1, 'S': 1, 'T': 2}
derivative_assignment: ((0,), ((0, 0),), ((0, 0), (0, 0)))
field_multiset: {'F': 1, 'S': 1, 'T': 1, 'Sp': 1}
derivative_assignment: ((0,), ((0, 0),), ((0, 0),), ((0, 0),))
field_multiset: {'F': 1, 'S': 1, 'Vp': 2}
derivative_assignment: ((0,), ((0, 0),), ((0, 0), (0, 0)))
field_multiset: {'F': 1, 'S': 1, 'Sp': 2}
derivative_assignment: ((0,), ((0, 0),), ((0, 0), (0, 0)))
field_multiset: {'F': 1, 'V': 2

derivative_assignment: ((0, 0), ((0, 0),), ((0, 1),))
field_multiset: {'D': 1, 'F': 2, 'S': 1, 'Vp': 1}
derivative_assignment: ((1, 0), ((0, 0),), ((0, 0),))
derivative_assignment: ((0, 0), ((1, 0),), ((0, 0),))
derivative_assignment: ((0, 0), ((0, 1),), ((0, 0),))
derivative_assignment: ((0, 0), ((0, 0),), ((1, 0),))
derivative_assignment: ((0, 0), ((0, 0),), ((0, 1),))
field_multiset: {'D': 1, 'F': 2, 'V': 1, 'T': 1}
derivative_assignment: ((1, 0), ((0, 0),), ((0, 0),))
derivative_assignment: ((0, 0), ((1, 0),), ((0, 0),))
derivative_assignment: ((0, 0), ((0, 1),), ((0, 0),))
derivative_assignment: ((0, 0), ((0, 0),), ((1, 0),))
derivative_assignment: ((0, 0), ((0, 0),), ((0, 1),))
field_multiset: {'D': 1, 'F': 2, 'V': 1, 'Sp': 1}
derivative_assignment: ((1, 0), ((0, 0),), ((0, 0),))
derivative_assignment: ((0, 0), ((1, 0),), ((0, 0),))
derivative_assignment: ((0, 0), ((0, 1),), ((0, 0),))
derivative_assignment: ((0, 0), ((0, 0),), ((1, 0),))
derivative_assignment: ((0, 0), ((0, 0),)

derivative_assignment: ((2,), ((1, 0),), ((0, 0),))
derivative_assignment: ((2,), ((0, 1),), ((0, 0),))
derivative_assignment: ((2,), ((0, 0),), ((1, 0),))
derivative_assignment: ((2,), ((0, 0),), ((0, 1),))
derivative_assignment: ((1,), ((2, 0),), ((0, 0),))
derivative_assignment: ((1,), ((0, 2),), ((0, 0),))
derivative_assignment: ((1,), ((1, 1),), ((0, 0),))
derivative_assignment: ((1,), ((0, 0),), ((2, 0),))
derivative_assignment: ((1,), ((0, 0),), ((0, 2),))
derivative_assignment: ((1,), ((0, 0),), ((1, 1),))
derivative_assignment: ((0,), ((2, 0),), ((1, 0),))
derivative_assignment: ((0,), ((2, 0),), ((0, 1),))
derivative_assignment: ((0,), ((0, 2),), ((1, 0),))
derivative_assignment: ((0,), ((0, 2),), ((0, 1),))
derivative_assignment: ((0,), ((1, 1),), ((1, 0),))
derivative_assignment: ((0,), ((1, 1),), ((0, 1),))
derivative_assignment: ((0,), ((1, 0),), ((2, 0),))
derivative_assignment: ((0,), ((1, 0),), ((0, 2),))
derivative_assignment: ((0,), ((1, 0),), ((1, 1),))
derivative_a

derivative_assignment: (((0, 3),), ((2, 0),))
derivative_assignment: (((0, 3),), ((0, 2),))
derivative_assignment: (((0, 3),), ((1, 1),))
derivative_assignment: (((2, 1),), ((2, 0),))
derivative_assignment: (((2, 1),), ((0, 2),))
derivative_assignment: (((2, 1),), ((1, 1),))
derivative_assignment: (((1, 2),), ((2, 0),))
derivative_assignment: (((1, 2),), ((0, 2),))
derivative_assignment: (((1, 2),), ((1, 1),))
derivative_assignment: (((2, 0),), ((3, 0),))
derivative_assignment: (((2, 0),), ((0, 3),))
derivative_assignment: (((2, 0),), ((2, 1),))
derivative_assignment: (((2, 0),), ((1, 2),))
derivative_assignment: (((0, 2),), ((3, 0),))
derivative_assignment: (((0, 2),), ((0, 3),))
derivative_assignment: (((0, 2),), ((2, 1),))
derivative_assignment: (((0, 2),), ((1, 2),))
derivative_assignment: (((1, 1),), ((3, 0),))
derivative_assignment: (((1, 1),), ((0, 3),))
derivative_assignment: (((1, 1),), ((2, 1),))
derivative_assignment: (((1, 1),), ((1, 2),))
field_multiset: {'D': 5, 'V': 1, '

derivative_assignment: ((2, 2), ((0, 0),))
derivative_assignment: ((0, 0), ((4, 0),))
derivative_assignment: ((0, 0), ((0, 4),))
derivative_assignment: ((0, 0), ((3, 1),))
derivative_assignment: ((0, 0), ((1, 3),))
derivative_assignment: ((0, 0), ((2, 2),))
derivative_assignment: ((3, 0), ((1, 0),))
derivative_assignment: ((3, 0), ((0, 1),))
derivative_assignment: ((2, 1), ((1, 0),))
derivative_assignment: ((2, 1), ((0, 1),))
derivative_assignment: ((1, 0), ((3, 0),))
derivative_assignment: ((1, 0), ((0, 3),))
derivative_assignment: ((1, 0), ((2, 1),))
derivative_assignment: ((1, 0), ((1, 2),))
derivative_assignment: ((2, 0), ((2, 0),))
derivative_assignment: ((2, 0), ((0, 2),))
derivative_assignment: ((2, 0), ((1, 1),))
derivative_assignment: ((1, 1), ((2, 0),))
derivative_assignment: ((1, 1), ((0, 2),))
derivative_assignment: ((1, 1), ((1, 1),))
field_multiset: {'D': 4, 'F': 2, 'Sp': 1}
derivative_assignment: ((4, 0), ((0, 0),))
derivative_assignment: ((3, 1), ((0, 0),))
derivative_a

...

LORENTZ CONTRACTIONS: 
{(2, 4): 2, (1, 3): 2, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(3, 4): 1, (2, 2): 1, (1, 4): 1, (1, 3): 1, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(2, 4): 1, (2, 3): 1, (1, 4): 1, (1, 3): 1, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(3, 3): 1, (2, 2): 1, (1, 4): 2, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(2, 3): 2, (1, 4): 2, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(4, 4): 1, (3, 3): 1, (2, 2): 1, (0, 1): 2}
LORENTZ CONTRACTIONS: 
{(3, 4): 2, (2, 2): 1, (0, 1): 2}
LORENTZ CONTRACTIONS: 
{(4, 4): 1, (2, 3): 2, (0, 1): 2}
LORENTZ CONTRACTIONS: 
{(3, 4): 1, (2, 4): 1, (2, 3): 1, (0, 1): 2}
LORENTZ CONTRACTIONS: 
{(3, 3): 1, (2, 4): 2, (0, 1): 2}
LORENTZ CONTRACTIONS: 
{(4, 4): 1, (3, 3): 1, (1, 2): 1, (0, 2): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(3, 4): 2, (1, 2): 1, (0, 2): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(4, 4): 1, (2, 3): 1, (1, 3): 1, (0, 2): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(3, 4): 1, (2, 4): 1, (1, 3): 1, (0, 2): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(3, 4): 1, (2, 3): 1, (1

LORENTZ CONTRACTIONS: 
{(1, 3): 2, (0, 2): 2}
LORENTZ CONTRACTIONS: 
{(2, 3): 1, (1, 1): 1, (0, 3): 1, (0, 2): 1}
LORENTZ CONTRACTIONS: 
{(1, 3): 1, (1, 2): 1, (0, 3): 1, (0, 2): 1}
LORENTZ CONTRACTIONS: 
{(2, 2): 1, (1, 1): 1, (0, 3): 2}
LORENTZ CONTRACTIONS: 
{(1, 2): 2, (0, 3): 2}

DERIVATIVE ASSIGNMENT: 
((2, 0), ((0, 2),))
LORENTZ INDEX GROUPING: 
[['DD', 'F'], ['F'], ['Sp', 'DD']]
[[[0, 1]], [[2]], [[3, 4]]]
LORENTZ CONTRACTIONS: 
{(4, 4): 1, (2, 2): 1, (1, 1): 1, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(2, 4): 2, (1, 1): 1, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(4, 4): 1, (1, 2): 2, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(2, 4): 1, (1, 4): 1, (1, 2): 1, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(2, 2): 1, (1, 4): 2, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(4, 4): 1, (2, 2): 1, (0, 1): 2}
LORENTZ CONTRACTIONS: 
{(2, 4): 2, (0, 1): 2}
LORENTZ CONTRACTIONS: 
{(4, 4): 1, (1, 2): 1, (0, 2): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(2, 4): 1, (1, 4): 1, (0, 2): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(2, 4): 1, (

{(2, 2): 1, (1, 3): 3, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(3, 3): 1, (2, 3): 1, (1, 2): 1, (0, 1): 2}
LORENTZ CONTRACTIONS: 
{(3, 3): 1, (2, 2): 1, (1, 3): 1, (0, 1): 2}
LORENTZ CONTRACTIONS: 
{(2, 3): 2, (1, 3): 1, (0, 1): 2}
LORENTZ CONTRACTIONS: 
{(3, 3): 1, (2, 3): 1, (1, 1): 1, (0, 2): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(3, 3): 1, (1, 3): 1, (1, 2): 1, (0, 2): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(2, 3): 1, (1, 3): 2, (0, 2): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(3, 3): 1, (2, 2): 1, (1, 1): 1, (0, 3): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(2, 3): 2, (1, 1): 1, (0, 3): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(3, 3): 1, (1, 2): 2, (0, 3): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(2, 3): 1, (1, 3): 1, (1, 2): 1, (0, 3): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(2, 2): 1, (1, 3): 2, (0, 3): 1, (0, 1): 1}
LORENTZ CONTRACTIONS: 
{(3, 3): 1, (1, 3): 1, (1, 1): 1, (0, 2): 2}
LORENTZ CONTRACTIONS: 
{(1, 3): 3, (0, 2): 2}
LORENTZ CONTRACTIONS: 
{(3, 3): 1, (1, 2): 1, (1, 1): 1, (0, 3): 1, (0,

LORENTZ CONTRACTIONS: 
{(1, 3): 1, (1, 2): 1, (0, 4): 2, (0, 3): 1}


CONSTITUENT FIELDS: 
{'D': 6, 'F': 1, 'Sp': 1}

DERIVATIVE ASSIGNMENT: 
((6,), ((0, 0),))
LORENTZ INDEX GROUPING: 
[['DDDDDD', 'F'], ['Sp']]
[[[0, 1]], [[2]]]
LORENTZ CONTRACTIONS: 
{(1, 1): 1, (0, 0): 3}
LORENTZ CONTRACTIONS: 
{(0, 1): 2, (0, 0): 2}

DERIVATIVE ASSIGNMENT: 
((0,), ((6, 0),))
LORENTZ INDEX GROUPING: 
[['F'], ['DDDDDD', 'Sp']]
[[[0]], [[1, 2]]]
LORENTZ CONTRACTIONS: 
{(1, 1): 3, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(1, 1): 2, (0, 1): 2}

DERIVATIVE ASSIGNMENT: 
((0,), ((0, 6),))
LORENTZ INDEX GROUPING: 
[['F'], ['Sp', 'DDDDDD']]
[[[0]], [[1, 2]]]
LORENTZ CONTRACTIONS: 
{(2, 2): 3, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(2, 2): 2, (0, 2): 2}

DERIVATIVE ASSIGNMENT: 
((0,), ((5, 1),))
LORENTZ INDEX GROUPING: 
[['F'], ['DDDDD', 'Sp', 'D']]
[[[0]], [[1, 2, 3]]]
LORENTZ CONTRACTIONS: 
{(1, 3): 1, (1, 1): 2, (0, 0): 1}
LORENTZ CONTRACTIONS: 
{(1, 3): 1, (1, 1): 1, (0, 1): 2}
LORENTZ CONTRACTIONS: 
{(1, 1): 2, (0,

{<frozendict {'F': 1, 'S': 3}>: {((0,),
   ((0, 0), (0, 0), (0, 0))): [{(0, 0): 1}]},
 <frozendict {'F': 1, 'S': 2, 'T': 1}>: {((0,),
   ((0, 0), (0, 0)),
   ((0, 0),)): [{(3, 3): 1, (0, 0): 1}, {(0, 3): 2}]},
 <frozendict {'F': 1, 'S': 2, 'Sp': 1}>: {((0,),
   ((0, 0), (0, 0)),
   ((0, 0),)): [{(0, 0): 1}]},
 <frozendict {'F': 1, 'S': 1, 'V': 2}>: {((0,),
   ((0, 0),),
   ((0, 0), (0, 0))): [{(2, 3): 1, (0, 0): 1}, {(0, 3): 1, (0, 2): 1}]},
 <frozendict {'F': 1, 'S': 1, 'V': 1, 'Vp': 1}>: {((0,),
   ((0, 0),),
   ((0, 0),),
   ((0, 0),)): [{(2, 3): 1, (0, 0): 1}, {(0, 3): 1, (0, 2): 1}]},
 <frozendict {'F': 1, 'S': 1, 'T': 2}>: {((0,),
   ((0, 0),),
   ((0, 0), (0, 0))): [{(3, 3): 1, (2, 2): 1, (0, 0): 1}, {(2, 3): 2,
    (0, 0): 1}, {(3, 3): 1, (0, 2): 2}, {(2, 3): 1, (0, 3): 1, (0, 2): 1}]},
 <frozendict {'F': 1, 'S': 1, 'T': 1, 'Sp': 1}>: {((0,),
   ((0, 0),),
   ((0, 0),),
   ((0, 0),)): [{(2, 2): 1, (0, 0): 1}, {(0, 2): 2}]},
 <frozendict {'F': 1, 'S': 1, 'Vp': 2}>: {((0,),
   ((

BUG! Indicating contractions with S fields. 

In [ ]:
massDim = 14
display_generate_operators(massDim, IBP=False)

In generate_operators(), field multisets generated
field_multiset: {'F': 1, 'S': 4}
derivative_assignment: ((0,), ((0, 0), (0, 0), (0, 0), (0, 0)))
field_multiset: {'F': 1, 'S': 3, 'T': 1}
derivative_assignment: ((0,), ((0, 0), (0, 0), (0, 0)), ((0, 0),))
field_multiset: {'F': 1, 'S': 3, 'Sp': 1}
derivative_assignment: ((0,), ((0, 0), (0, 0), (0, 0)), ((0, 0),))
field_multiset: {'F': 1, 'S': 2, 'V': 2}
derivative_assignment: ((0,), ((0, 0), (0, 0)), ((0, 0), (0, 0)))
field_multiset: {'F': 1, 'S': 2, 'V': 1, 'Vp': 1}
derivative_assignment: ((0,), ((0, 0), (0, 0)), ((0, 0),), ((0, 0),))
field_multiset: {'F': 1, 'S': 2, 'T': 2}
derivative_assignment: ((0,), ((0, 0), (0, 0)), ((0, 0), (0, 0)))
field_multiset: {'F': 1, 'S': 2, 'T': 1, 'Sp': 1}
derivative_assignment: ((0,), ((0, 0), (0, 0)), ((0, 0),), ((0, 0),))
field_multiset: {'F': 1, 'S': 2, 'Vp': 2}
derivative_assignment: ((0,), ((0, 0), (0, 0)), ((0, 0), (0, 0)))
field_multiset: {'F': 1, 'S': 2, 'Sp': 2}
derivative_assignment: ((0,), (

derivative_assignment: (((2, 0), (0, 0)), ((0, 0),), ((0, 0),))
derivative_assignment: (((0, 2), (0, 0)), ((0, 0),), ((0, 0),))
derivative_assignment: (((1, 1), (0, 0)), ((0, 0),), ((0, 0),))
derivative_assignment: (((1, 0), (1, 0)), ((0, 0),), ((0, 0),))
derivative_assignment: (((1, 0), (0, 1)), ((0, 0),), ((0, 0),))
derivative_assignment: (((0, 1), (0, 1)), ((0, 0),), ((0, 0),))
derivative_assignment: (((0, 0), (0, 0)), ((2, 0),), ((0, 0),))
derivative_assignment: (((0, 0), (0, 0)), ((0, 2),), ((0, 0),))
derivative_assignment: (((0, 0), (0, 0)), ((1, 1),), ((0, 0),))
derivative_assignment: (((0, 0), (0, 0)), ((0, 0),), ((2, 0),))
derivative_assignment: (((0, 0), (0, 0)), ((0, 0),), ((0, 2),))
derivative_assignment: (((0, 0), (0, 0)), ((0, 0),), ((1, 1),))
derivative_assignment: (((1, 0), (0, 0)), ((1, 0),), ((0, 0),))
derivative_assignment: (((1, 0), (0, 0)), ((0, 1),), ((0, 0),))
derivative_assignment: (((0, 1), (0, 0)), ((1, 0),), ((0, 0),))
derivative_assignment: (((0, 1), (0, 0))

derivative_assignment: (((0, 0),), ((1, 0),), ((0, 1), (0, 0)))
derivative_assignment: (((0, 0),), ((0, 1),), ((1, 0), (0, 0)))
derivative_assignment: (((0, 0),), ((0, 1),), ((0, 1), (0, 0)))
field_multiset: {'D': 2, 'S': 1, 'T': 1, 'Sp': 2}
derivative_assignment: (((2, 0),), ((0, 0),), ((0, 0), (0, 0)))
derivative_assignment: (((0, 2),), ((0, 0),), ((0, 0), (0, 0)))
derivative_assignment: (((1, 1),), ((0, 0),), ((0, 0), (0, 0)))
derivative_assignment: (((0, 0),), ((2, 0),), ((0, 0), (0, 0)))
derivative_assignment: (((0, 0),), ((0, 2),), ((0, 0), (0, 0)))
derivative_assignment: (((0, 0),), ((1, 1),), ((0, 0), (0, 0)))
derivative_assignment: (((0, 0),), ((0, 0),), ((2, 0), (0, 0)))
derivative_assignment: (((0, 0),), ((0, 0),), ((0, 2), (0, 0)))
derivative_assignment: (((0, 0),), ((0, 0),), ((1, 1), (0, 0)))
derivative_assignment: (((0, 0),), ((0, 0),), ((1, 0), (1, 0)))
derivative_assignment: (((0, 0),), ((0, 0),), ((1, 0), (0, 1)))
derivative_assignment: (((0, 0),), ((0, 0),), ((0, 1),

derivative_assignment: (((0, 0),), ((1, 0), (0, 1)), ((0, 0),))
derivative_assignment: (((0, 0),), ((0, 1), (0, 1)), ((0, 0),))
derivative_assignment: (((0, 0),), ((0, 0), (0, 0)), ((2, 0),))
derivative_assignment: (((0, 0),), ((0, 0), (0, 0)), ((0, 2),))
derivative_assignment: (((0, 0),), ((0, 0), (0, 0)), ((1, 1),))
derivative_assignment: (((1, 0),), ((1, 0), (0, 0)), ((0, 0),))
derivative_assignment: (((1, 0),), ((0, 1), (0, 0)), ((0, 0),))
derivative_assignment: (((0, 1),), ((1, 0), (0, 0)), ((0, 0),))
derivative_assignment: (((0, 1),), ((0, 1), (0, 0)), ((0, 0),))
derivative_assignment: (((1, 0),), ((0, 0), (0, 0)), ((1, 0),))
derivative_assignment: (((1, 0),), ((0, 0), (0, 0)), ((0, 1),))
derivative_assignment: (((0, 1),), ((0, 0), (0, 0)), ((1, 0),))
derivative_assignment: (((0, 1),), ((0, 0), (0, 0)), ((0, 1),))
derivative_assignment: (((0, 0),), ((1, 0), (0, 0)), ((1, 0),))
derivative_assignment: (((0, 0),), ((1, 0), (0, 0)), ((0, 1),))
derivative_assignment: (((0, 0),), ((0, 

derivative_assignment: (((0, 0),), ((0, 0), (0, 0)), ((2, 0),))
derivative_assignment: (((0, 0),), ((0, 0), (0, 0)), ((0, 2),))
derivative_assignment: (((0, 0),), ((0, 0), (0, 0)), ((1, 1),))
derivative_assignment: (((1, 0),), ((1, 0), (0, 0)), ((0, 0),))
derivative_assignment: (((1, 0),), ((0, 1), (0, 0)), ((0, 0),))
derivative_assignment: (((0, 1),), ((1, 0), (0, 0)), ((0, 0),))
derivative_assignment: (((0, 1),), ((0, 1), (0, 0)), ((0, 0),))
derivative_assignment: (((1, 0),), ((0, 0), (0, 0)), ((1, 0),))
derivative_assignment: (((1, 0),), ((0, 0), (0, 0)), ((0, 1),))
derivative_assignment: (((0, 1),), ((0, 0), (0, 0)), ((1, 0),))
derivative_assignment: (((0, 1),), ((0, 0), (0, 0)), ((0, 1),))
derivative_assignment: (((0, 0),), ((1, 0), (0, 0)), ((1, 0),))
derivative_assignment: (((0, 0),), ((1, 0), (0, 0)), ((0, 1),))
derivative_assignment: (((0, 0),), ((0, 1), (0, 0)), ((1, 0),))
derivative_assignment: (((0, 0),), ((0, 1), (0, 0)), ((0, 1),))
field_multiset: {'D': 2, 'T': 1, 'Sp': 3

...

derivative_assignment: ((0,), ((2, 0), (1, 0), (0, 0)))
derivative_assignment: ((0,), ((2, 0), (0, 1), (0, 0)))
derivative_assignment: ((0,), ((0, 2), (1, 0), (0, 0)))
derivative_assignment: ((0,), ((0, 2), (0, 1), (0, 0)))
derivative_assignment: ((0,), ((1, 1), (1, 0), (0, 0)))
derivative_assignment: ((0,), ((1, 1), (0, 1), (0, 0)))
derivative_assignment: ((0,), ((1, 0), (1, 0), (1, 0)))
derivative_assignment: ((0,), ((1, 0), (1, 0), (0, 1)))
derivative_assignment: ((0,), ((1, 0), (0, 1), (0, 1)))
derivative_assignment: ((0,), ((0, 1), (0, 1), (0, 1)))
derivative_assignment: ((2,), ((1, 0), (0, 0), (0, 0)))
derivative_assignment: ((2,), ((0, 1), (0, 0), (0, 0)))
derivative_assignment: ((1,), ((2, 0), (0, 0), (0, 0)))
derivative_assignment: ((1,), ((0, 2), (0, 0), (0, 0)))
derivative_assignment: ((1,), ((1, 1), (0, 0), (0, 0)))
derivative_assignment: ((1,), ((1, 0), (1, 0), (0, 0)))
derivative_assignment: ((1,), ((1, 0), (0, 1), (0, 0)))
derivative_assignment: ((1,), ((0, 1), (0, 1), (

derivative_assignment: ((0,), ((1, 1),), ((1, 0),), ((0, 0),))
derivative_assignment: ((0,), ((1, 1),), ((0, 1),), ((0, 0),))
derivative_assignment: ((0,), ((2, 0),), ((0, 0),), ((1, 0),))
derivative_assignment: ((0,), ((2, 0),), ((0, 0),), ((0, 1),))
derivative_assignment: ((0,), ((0, 2),), ((0, 0),), ((1, 0),))
derivative_assignment: ((0,), ((0, 2),), ((0, 0),), ((0, 1),))
derivative_assignment: ((0,), ((1, 1),), ((0, 0),), ((1, 0),))
derivative_assignment: ((0,), ((1, 1),), ((0, 0),), ((0, 1),))
derivative_assignment: ((0,), ((1, 0),), ((2, 0),), ((0, 0),))
derivative_assignment: ((0,), ((1, 0),), ((0, 2),), ((0, 0),))
derivative_assignment: ((0,), ((1, 0),), ((1, 1),), ((0, 0),))
derivative_assignment: ((0,), ((0, 1),), ((2, 0),), ((0, 0),))
derivative_assignment: ((0,), ((0, 1),), ((0, 2),), ((0, 0),))
derivative_assignment: ((0,), ((0, 1),), ((1, 1),), ((0, 0),))
derivative_assignment: ((0,), ((1, 0),), ((0, 0),), ((2, 0),))
derivative_assignment: ((0,), ((1, 0),), ((0, 0),), ((0

derivative_assignment: ((0,), ((3, 0), (0, 0)), ((0, 0),))
derivative_assignment: ((0,), ((0, 3), (0, 0)), ((0, 0),))
derivative_assignment: ((0,), ((2, 1), (0, 0)), ((0, 0),))
derivative_assignment: ((0,), ((1, 2), (0, 0)), ((0, 0),))
derivative_assignment: ((0,), ((2, 0), (1, 0)), ((0, 0),))
derivative_assignment: ((0,), ((2, 0), (0, 1)), ((0, 0),))
derivative_assignment: ((0,), ((0, 2), (1, 0)), ((0, 0),))
derivative_assignment: ((0,), ((0, 2), (0, 1)), ((0, 0),))
derivative_assignment: ((0,), ((1, 1), (1, 0)), ((0, 0),))
derivative_assignment: ((0,), ((1, 1), (0, 1)), ((0, 0),))
derivative_assignment: ((0,), ((0, 0), (0, 0)), ((3, 0),))
derivative_assignment: ((0,), ((0, 0), (0, 0)), ((0, 3),))
derivative_assignment: ((0,), ((0, 0), (0, 0)), ((2, 1),))
derivative_assignment: ((0,), ((0, 0), (0, 0)), ((1, 2),))
derivative_assignment: ((2,), ((1, 0), (0, 0)), ((0, 0),))
derivative_assignment: ((2,), ((0, 1), (0, 0)), ((0, 0),))
derivative_assignment: ((2,), ((0, 0), (0, 0)), ((1, 0),

derivative_assignment: ((0,), ((1, 1), (0, 1), (0, 0)))
derivative_assignment: ((0,), ((1, 0), (1, 0), (1, 0)))
derivative_assignment: ((0,), ((1, 0), (1, 0), (0, 1)))
derivative_assignment: ((0,), ((1, 0), (0, 1), (0, 1)))
derivative_assignment: ((0,), ((0, 1), (0, 1), (0, 1)))
derivative_assignment: ((2,), ((1, 0), (0, 0), (0, 0)))
derivative_assignment: ((2,), ((0, 1), (0, 0), (0, 0)))
derivative_assignment: ((1,), ((2, 0), (0, 0), (0, 0)))
derivative_assignment: ((1,), ((0, 2), (0, 0), (0, 0)))
derivative_assignment: ((1,), ((1, 1), (0, 0), (0, 0)))
derivative_assignment: ((1,), ((1, 0), (1, 0), (0, 0)))
derivative_assignment: ((1,), ((1, 0), (0, 1), (0, 0)))
derivative_assignment: ((1,), ((0, 1), (0, 1), (0, 0)))
field_multiset: {'D': 3, 'F': 1, 'Vp': 1, 'Sp': 2}
derivative_assignment: ((3,), ((0, 0),), ((0, 0), (0, 0)))
derivative_assignment: ((0,), ((3, 0),), ((0, 0), (0, 0)))
derivative_assignment: ((0,), ((0, 3),), ((0, 0), (0, 0)))
derivative_assignment: ((0,), ((2, 1),), ((0

derivative_assignment: ((1, 0, 0), ((1, 0), (0, 0)))
derivative_assignment: ((1, 0, 0), ((0, 1), (0, 0)))
field_multiset: {'D': 2, 'F': 3, 'T': 1, 'Sp': 1}
derivative_assignment: ((2, 0, 0), ((0, 0),), ((0, 0),))
derivative_assignment: ((1, 1, 0), ((0, 0),), ((0, 0),))
derivative_assignment: ((0, 0, 0), ((2, 0),), ((0, 0),))
derivative_assignment: ((0, 0, 0), ((0, 2),), ((0, 0),))
derivative_assignment: ((0, 0, 0), ((1, 1),), ((0, 0),))
derivative_assignment: ((0, 0, 0), ((0, 0),), ((2, 0),))
derivative_assignment: ((0, 0, 0), ((0, 0),), ((0, 2),))
derivative_assignment: ((0, 0, 0), ((0, 0),), ((1, 1),))
derivative_assignment: ((1, 0, 0), ((1, 0),), ((0, 0),))
derivative_assignment: ((1, 0, 0), ((0, 1),), ((0, 0),))
derivative_assignment: ((1, 0, 0), ((0, 0),), ((1, 0),))
derivative_assignment: ((1, 0, 0), ((0, 0),), ((0, 1),))
derivative_assignment: ((0, 0, 0), ((1, 0),), ((1, 0),))
derivative_assignment: ((0, 0, 0), ((1, 0),), ((0, 1),))
derivative_assignment: ((0, 0, 0), ((0, 1),), 

## TO DO:
- DONE Fix bug where contractions are on S fields. 
- Remove derivative assignments with same number on \bar{psi}
- Remove contractions with self contractions on F or T
- implement C,P,T
- implement EOM